In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mpmath
import math

In [2]:
def autocov_est(vhat,j):
    if j >=0:
        T = len(vhat)
        v_t = vhat[j:T]
        v_tm1 = vhat[0:T-j]
        gamma_hat_j = np.dot(v_t.T,v_tm1)/T
        return gamma_hat_j
    else:
        T = len(vhat)
        v_t = vhat[-j:T]
        v_tm1 = vhat[0:T+j]
        gamma_hat_j = np.dot(v_t.T,v_tm1)/T
        gamma_hat_j_T = gamma_hat_j.T
        return gamma_hat_j_T

In [3]:
def NW_band(vhat,c,p,q=2):
    n = len(vhat)
    T = len(vhat)
    if vhat.shape[1] == 1:
        nw_lag = c*(n**p)
        a_n = nw_lag
        f_q = 0
        f_0 = 0
        for j in range(-int(a_n),int(a_n)+1):
            f_q = f_q + (np.abs(j)**q)*autocov_est(vhat,j)
            f_0 = f_0 + autocov_est(vhat,j)

        a_hat_2 = (f_q/f_0)**2
        ST_NW = 2.6614*(T*a_hat_2)**(0.2) #bandwidth
        ST_NW = np.float(ST_NW)
        return ST_NW
    elif vhat.shape[1] == 2:
        nw_lag = c*(n**p)
        a_n = nw_lag
        f_q = 0
        f_0 = 0
        wp = np.array([[0,1]])
        w = wp.T
        for j in range(-int(a_n),int(a_n)+1):
            f_q = f_q + (np.abs(j)**q)*(np.dot(np.dot(wp,autocov_est(vhat,j)),w))
            f_0 = f_0 + (np.dot(np.dot(wp,autocov_est(vhat,j)),w))

        a_hat_2 = (f_q/f_0)**2
        ST_NW = 2.6614*(T*a_hat_2)**(0.2) #bandwidth
        ST_NW = np.float(ST_NW)
        return ST_NW

In [4]:
def AD_band(vhat):
    vhat = np.copy(vhat)
    #vhat = np.copy(xuhat)
    if vhat.shape[1] == 1:
        pass
    elif vhat.shape[1] == 2:
        vhat = np.copy(np.reshape(vhat[:,1],(len(vhat[:,1]),1))) 
    
    T = len(vhat)

    vhat_t = np.copy(vhat[1:])
    vhat_tm1 = np.copy(vhat[:-1])
    y2 = np.copy(vhat_t)
    X2 = np.copy(vhat_tm1)
    rho_hat = np.linalg.inv(X2.transpose().dot(X2)).dot(X2.transpose()).dot(y2) #without constant
    #print (rho_hat)
    #if (np.abs(rho_hat-1) < 0.001) & (rho_hat>=1):
    #    rho_hat = 1.001
    #elif (np.abs(rho_hat-1) < 0.001) & (rho_hat<1):
    #    rho_hat = 0.999
        
    if np.abs(rho_hat) >= 0.97:
        rho_hat = 0.97*np.sign(rho_hat)

    alpha_hat_2 = (4*(rho_hat)**2)/((1-rho_hat)**4) #univariate version CLP
    ST = 2.6614*(T*alpha_hat_2)**(0.2) #bandwidth
    ST = np.float(ST)
    return (ST)

In [5]:
def noncen_chisq(k,j,x):
    #k is df
    #j is non-centrality parameter
    pdf_v = 0.5*np.exp(-0.5*(x+j))*((x/j)**(k/4-0.5))*mpmath.besseli(0.5*k-1, np.sqrt(j*x), derivative=0)
    return pdf_v

In [6]:
def chisq_dfone(x):
    pdf_v = (np.exp(-x/2))/(np.sqrt(2*math.pi*x))
    return pdf_v

In [7]:
def SPJ_band(vhat,w,z_a=1.96,delta=2,q=2,g=6,c=0.539):
    #z_a = 1.96 # w is tuning parameter
    vhat = np.copy(vhat)
    #vhat = np.copy(xuhat)
    if vhat.shape[1] == 1:
        pass
    elif vhat.shape[1] == 2:
        vhat = np.copy(np.reshape(vhat[:,1],(len(vhat[:,1]),1))) 
    
    T = len(vhat)

    vhat_t = np.copy(vhat[1:])
    vhat_tm1 = np.copy(vhat[:-1])
    y2 = np.copy(vhat_t)
    X2 = np.copy(vhat_tm1)
    rho_hat = np.linalg.inv(X2.transpose().dot(X2)).dot(X2.transpose()).dot(y2) #without constant
    
    if np.abs(rho_hat) >= 0.97:
        rho_hat = 0.97*np.sign(rho_hat)
    
    d_hat = (2*rho_hat)/(1-rho_hat)**2
    d_hat = np.float(d_hat)
    x_val = z_a**2
    
    ### calculation of b_hat
    G_0 = chisq_dfone(x=x_val)
    G_d = noncen_chisq(k=1,j=delta**2,x=x_val)
    k_d = ((delta**2)/(2*x_val))*noncen_chisq(k=3,j=delta**2,x=x_val)
    if d_hat*(w*G_0 - G_d) > 0:
        b_hat = (((q*g*d_hat*(w*G_0 - G_d))/(c*x_val*k_d))**(1/3))*(T**(-2/3))
        #print (b_hat)
    elif d_hat*(w*G_0 - G_d) <= 0:
        b_hat = np.log(T)/T
    else:
        raise Exception("Error")
    spj_band = b_hat * T
    spj_band = np.float(spj_band)
    return spj_band

In [9]:
def Parzen(x):
    if 0<=np.abs(x)<=0.5:
        kx = 1-6*(x**2)+6*((np.abs(x))**3)
        return kx
    elif 0.5<=np.abs(x)<=1:
        kx = 2*((1-np.abs(x))**3)
        return kx
    else:
        kx = 0
        return kx

In [10]:
def LRV(vhat,c,p,q=2,band="NW"):
    if band == "NW":
        n = len(vhat)                                
        LRV = 0
        b_n = NW_band(vhat,c,p,q=2) # band is NW
        for j in range(-int(n),int(n)+1):
            LRV = LRV + Parzen(j/b_n)*autocov_est(vhat,j)
        return LRV    
    elif band == "AD":
        n = len(vhat)                                
        LRV = 0
        b_n = AD_band(vhat) #band is AD
        for j in range(-int(n),int(n)+1):
            LRV = LRV + Parzen(j/b_n)*autocov_est(vhat,j)
        return LRV
    elif band == "SPJ":
        n = len(vhat)                                
        LRV = 0
        b_n = SPJ_band(vhat,w=5,z_a=1.96,delta=2,q=2,g=6,c=0.539) #band is AD
        for j in range(-int(n),int(n)+1):
            LRV = LRV + Parzen(j/b_n)*autocov_est(vhat,j)
        return LRV

In [11]:
def G_test(R,r,b,X,vhat,c,p,q=2,band="NW"):
    if band == "NW":
        j = np.dot(R,b)-r
        Xv = np.linalg.inv(np.dot(X.T,X))
        s = np.dot(np.dot(R,Xv),R.transpose())

        G_pre = np.dot(np.dot(j.transpose(),np.linalg.inv(s)),j)
        G_stat = G_pre/LRV(vhat,c,p,q=2,band="NW")
        G_stat = np.float(G_stat)
        return G_stat
    elif band == "AD":
        j = np.dot(R,b)-r
        #s = np.dot(np.dot(R,1/np.sum(x*x)),R.transpose())
        Xv = np.linalg.inv(np.dot(X.T,X))
        s = np.dot(np.dot(R,Xv),R.transpose())

        G_pre = np.dot(np.dot(j.transpose(),np.linalg.inv(s)),j)
        G_stat = G_pre/LRV(vhat,c,p,q=2,band="AD")
        G_stat = np.float(G_stat)
        return G_stat
    elif band == "SPJ":
        j = np.dot(R,b)-r
        #s = np.dot(np.dot(R,1/np.sum(x*x)),R.transpose())
        Xv = np.linalg.inv(np.dot(X.T,X))
        s = np.dot(np.dot(R,Xv),R.transpose())

        G_pre = np.dot(np.dot(j.transpose(),np.linalg.inv(s)),j)
        G_stat = G_pre/LRV(vhat,c,p,q=2,band="SPJ")
        G_stat = np.float(G_stat)
        return G_stat

In [12]:
def H_test(R,r,b,X,vhat,c,p,q=2,band="NW"):
    if band == "NW":
        n = len(vhat)
        j = np.dot(R,b)-r
        Xv = np.linalg.inv(np.dot(X.T,X))
        s = np.dot(R,Xv)
        nomega = n*LRV(vhat,c,p,q=2,band="NW")
        center = np.dot(np.dot(s,nomega),s.transpose())

        H_stat = np.dot(np.dot(j.transpose(),np.linalg.inv(center)),j)
        H_stat = np.float(H_stat)
        return H_stat
        
    elif band == "AD":
        n = len(vhat)
        j = np.dot(R,b)-r
        Xv = np.linalg.inv(np.dot(X.T,X))
        s = np.dot(R,Xv)
        nomega = n*LRV(vhat,c,p,q=2,band="AD")
        center = np.dot(np.dot(s,nomega),s.transpose())

        H_stat = np.dot(np.dot(j.transpose(),np.linalg.inv(center)),j)
        H_stat = np.float(H_stat)
        return H_stat
    
    elif band == "SPJ":
        n = len(vhat)
        j = np.dot(R,b)-r
        Xv = np.linalg.inv(np.dot(X.T,X))
        s = np.dot(R,Xv)
        nomega = n*LRV(vhat,c,p,q=2,band="SPJ")
        center = np.dot(np.dot(s,nomega),s.transpose())

        H_stat = np.dot(np.dot(j.transpose(),np.linalg.inv(center)),j)
        H_stat = np.float(H_stat)
        return H_stat

In [13]:
def OU_p(t_0,t_end,length,k_u,mu,sigma_u):
    
    #nprn = np.random.seed(i+10)
    t = np.linspace(t_0,t_end,length) # define time axis
    dt = np.mean(np.diff(t))
    u = np.zeros(length)
    u0 = np.random.normal(loc=0.0,scale=1.0) # initial condition
    u[0] = 0

    drift = lambda u,t: k_u*(mu-u) # define drift term, google to learn about lambda
    diffusion = lambda u,t: sigma_u # define diffusion term
    noise = np.random.normal(loc=0.0,scale=1.0,size=length)*np.sqrt(dt) #define noise process

    # solve SDE
    for i in range(1,length):
        u[i] = u[i-1] + drift(u[i-1],i*dt)*dt + diffusion(u[i-1],i*dt)*noise[i]

    u = np.reshape(u, (len(u), 1))
    return u

In [14]:
def OU_AR(t_0,t_end,length,k_u,mu,sigma_u):
    
    #nprn = np.random.seed(i+10)
    t = np.linspace(t_0,t_end,length) # define time axis
    dt = np.mean(np.diff(t))
    
    intercept = k_u * mu * dt
    rho_u = -1 * (k_u*dt - 1)
    #eps_std = sigma_u * np.sqrt(dt)
    
    u = np.zeros(length)
    #u0 = np.random.normal(loc=0.0,scale=1.0) # initial condition
    u[0] = 0
    
    noise = np.random.normal(loc=0.0,scale=1.0,size=length)
    
    for i in range(1,length):
        u[i] = intercept + rho_u*u[i-1] + sigma_u * np.sqrt(dt) * noise[i]
    #print (rho_u, intercept)
        
    u = np.reshape(u, (len(u), 1))
    return u

In [51]:
c1 = 0.26880
c2 = 0.98804
c3 = -0.04904
c4 = 5.13641
c5 = -2.37697
c6 = 0.15189
c7 = -2.17925
c8 = 3.06497
c9 = 0.22194
ku = np.linspace(0,6,5) + 0
ku = np.insert(ku, 1, 0.5)
ku = np.delete(ku, 4)
kx = np.linspace(0,6,5) + 0
kx = np.insert(kx, 1, 0.5)
kx = np.delete(kx, 4)
kx[2] = kx[2] + 0.0718
ku[2] = ku[2] + 0.0718
table_mean_AD = np.zeros((5,5))
table_mean_SPJ = np.zeros((5,5))
table_med_AD = np.zeros((5,5))
table_med_SPJ = np.zeros((5,5))
table_min_AD = np.zeros((5,5))
table_min_SPJ = np.zeros((5,5))
table_max_AD = np.zeros((5,5))
table_max_SPJ = np.zeros((5,5))

table_rej_AD = np.zeros((5,5))
table_rej_SPJ = np.zeros((5,5))

table_rej_AD_fixed = np.zeros((5,5))
table_rej_SPJ_fixed = np.zeros((5,5))

table_avgband_AD = np.zeros((5,5))
table_avgband_SPJ = np.zeros((5,5))

table_stdband_AD = np.zeros((5,5))
table_stdband_SPJ = np.zeros((5,5))

table_mean_NW1 = np.zeros((5,5))
table_mean_NW2 = np.zeros((5,5))
table_med_NW1 = np.zeros((5,5))
table_med_NW2 = np.zeros((5,5))
table_min_NW1 = np.zeros((5,5))
table_min_NW2 = np.zeros((5,5))
table_max_NW1 = np.zeros((5,5))
table_max_NW2 = np.zeros((5,5))

table_rej_NW1 = np.zeros((5,5))
table_rej_NW2 = np.zeros((5,5))

table_rej_NW1_fixed = np.zeros((5,5))
table_rej_NW2_fixed = np.zeros((5,5))

table_avgband_NW1 = np.zeros((5,5))
table_avgband_NW2 = np.zeros((5,5))

table_stdband_NW1 = np.zeros((5,5))
table_stdband_NW2 = np.zeros((5,5))


AD_set = [0]
SPJ_set = [0]
NW1_set = [0]
NW2_set = [0]

for w in range(len(ku)):
    for d in range(len(kx)):
        AD_set = [0]
        SPJ_set = [0]
        NW1_set = [0]
        NW2_set = [0]
        
        name1 = "band_AD"+str(w)+str(d)
        name2 = "band_SPJ"+str(w)+str(d)
        name1_n1 = "band_NW1"+str(w)+str(d)
        name1_n2 = "band_NW2"+str(w)+str(d)
        globals()[name1] = [0]
        globals()[name2] = [0]
        globals()[name1_n1] = [0]
        globals()[name1_n2] = [0]
        
        newname1 = "fixed_AD"+str(w)+str(d)
        newname2 = "fixed_SPJ"+str(w)+str(d)
        newname1_n1 = "fixed_NW1"+str(w)+str(d)
        newname1_n2 = "fixed_NW2"+str(w)+str(d)
        globals()[newname1] = [0]
        globals()[newname2] = [0]
        globals()[newname1_n1] = [0]
        globals()[newname1_n2] = [0]
        
        name3 = "stat_AD"+str(w)+str(d)
        name4 = "stat_SPJ"+str(w)+str(d)
        name3_n1 = "stat_NW1"+str(w)+str(d)
        name3_n2 = "stat_NW2"+str(w)+str(d)
        globals()[name3] = [0]
        globals()[name4] = [0]
        globals()[name3_n1] = [0]
        globals()[name3_n2] = [0]
        
        for i in range(200):
            np.random.seed(i)
            u = OU_p(0,30,30*252,ku[w],0,0.0097)
            x = OU_p(0,30,30*252,kx[d],0,0.0998)
            
            #u = OU_AR(0,30,50,ku[w],0,0.0097)
            #x = OU_AR(0,30,50,kx[d],0,0.0998)
            
            #u = np.copy(u[::63]) #quaterly
            #x = np.copy(x[::63])
            Tnum = x.shape[0]
            
            b0 = 0
            b1 = 1
            y = b0 + b1*x + u
            nu = len(x)
            cons = np.ones((nu,1))
            X = np.hstack((cons,x))
            coeffs = np.linalg.inv(X.transpose().dot(X)).dot(X.transpose()).dot(y)

            X_hat = np.dot(X,coeffs)
            uhat = y - X_hat
            vhat = np.copy(uhat) #G-test
            #vhat = np.copy(X*uhat) #H-test

            R = np.array([[1, 0], [0, 1]])
            r = np.array([[0],[1]])
            
            #####################################################################
            
            vhat2 = np.copy(vhat)
            AD1 = G_test(R,r,coeffs,X,vhat2,c=1,p=4/25,q=2,band="AD")
            #AD1 = H_test(R,r,coeffs,X,vhat,c=1,p=4/25,q=2,band="AD")
            AD_set.append(AD1)
            globals()[name3].append(AD1)
            vhat2 = np.copy(vhat)
            AD_b = AD_band(vhat2)
            globals()[name1].append(AD_b/Tnum)
            
            AD_fixed_cv = 5.9915 + c1*(AD_b/Tnum*5.9915) + c2*((AD_b/Tnum)*(5.9915**2)) + c3*((AD_b/Tnum)*(5.9915**3)) + c4*(((AD_b/Tnum)**2)*5.9915) + c5*(((AD_b/Tnum)**2)*(5.9915**2)) + c6*(((AD_b/Tnum)**2)*(5.9915**3)) + c7*(((AD_b/Tnum)**3)*(5.9915**1)) + c8*(((AD_b/Tnum)**3)*(5.9915**2)) + c9*(((AD_b/Tnum)**3)*(5.9915**3))
            globals()[newname1].append(AD_fixed_cv)
            
            #####################################################################
            vhat3 = np.copy(vhat)
            SPJ1 = G_test(R,r,coeffs,X,vhat3,c=8.5*((1/100)**(21/25)),p=21/25,q=2,band="SPJ")
            #SPJ1 = H_test(R,r,coeffs,X,vhat,c=8.5*((1/100)**(21/25)),p=21/25,q=2,band="SPJ")
            SPJ_set.append(SPJ1)
            globals()[name4].append(SPJ1)
            vhat3 = np.copy(vhat)
            SPJ_b = SPJ_band(vhat3,w=5,z_a=1.96,delta=2,q=2,g=6,c=0.539) #w is tuning parameter
            globals()[name2].append(SPJ_b/Tnum)
            
            SPJ_fixed_cv = 5.9915 + c1*(SPJ_b/Tnum*5.9915) + c2*((SPJ_b/Tnum)*(5.9915**2)) + c3*((SPJ_b/Tnum)*(5.9915**3)) + c4*(((SPJ_b/Tnum)**2)*5.9915) + c5*(((SPJ_b/Tnum)**2)*(5.9915**2)) + c6*(((SPJ_b/Tnum)**2)*(5.9915**3)) + c7*(((SPJ_b/Tnum)**3)*(5.9915**1)) + c8*(((SPJ_b/Tnum)**3)*(5.9915**2)) + c9*(((SPJ_b/Tnum)**3)*(5.9915**3))
            globals()[newname2].append(SPJ_fixed_cv)
            
            #####################################################################
            vhat3 = np.copy(vhat)
            NW1 = G_test(R,r,coeffs,X,vhat3,c=4*((1/100)**(4/25)),p=4/25,q=2,band="NW")
            #NW1 = H_test(R,r,coeffs,X,vhat3,c=4*((1/100)**(4/25)),p=4/25,q=2,band="NW")
            NW1_set.append(NW1)
            globals()[name3_n1].append(NW1)
            vhat3 = np.copy(vhat)
            NW1_b = NW_band(vhat3,c=4*((1/100)**(4/25)),p=4/25,q=2)
            globals()[name1_n1].append(NW1_b/Tnum)
            
            NW1_fixed_cv = 5.9915 + c1*(NW1_b/Tnum*5.9915) + c2*((NW1_b/Tnum)*(5.9915**2)) + c3*((NW1_b/Tnum)*(5.9915**3)) + c4*(((NW1_b/Tnum)**2)*5.9915) + c5*(((NW1_b/Tnum)**2)*(5.9915**2)) + c6*(((NW1_b/Tnum)**2)*(5.9915**3)) + c7*(((NW1_b/Tnum)**3)*(5.9915**1)) + c8*(((NW1_b/Tnum)**3)*(5.9915**2)) + c9*(((NW1_b/Tnum)**3)*(5.9915**3))
            globals()[newname1_n1].append(NW1_fixed_cv)
            
            #####################################################################
            vhat3 = np.copy(vhat)
            NW2 = G_test(R,r,coeffs,X,vhat3,c=8.5*((1/100)**(21/25)),p=21/25,q=2,band="NW")
            #NW2 = H_test(R,r,coeffs,X,vhat3,c=4*((1/100)**(4/25)),p=4/25,q=2,band="NW")
            NW2_set.append(NW2)
            globals()[name3_n2].append(NW2)
            vhat3 = np.copy(vhat)
            NW2_b = NW_band(vhat3,c=8.5*((1/100)**(21/25)),p=21/25,q=2)
            globals()[name1_n2].append(NW2_b/Tnum)
            
            NW2_fixed_cv = 5.9915 + c1*(NW2_b/Tnum*5.9915) + c2*((NW2_b/Tnum)*(5.9915**2)) + c3*((NW2_b/Tnum)*(5.9915**3)) + c4*(((NW2_b/Tnum)**2)*5.9915) + c5*(((NW2_b/Tnum)**2)*(5.9915**2)) + c6*(((NW2_b/Tnum)**2)*(5.9915**3)) + c7*(((NW2_b/Tnum)**3)*(5.9915**1)) + c8*(((NW2_b/Tnum)**3)*(5.9915**2)) + c9*(((NW2_b/Tnum)**3)*(5.9915**3))
            globals()[newname1_n2].append(NW2_fixed_cv)
            
        table_med_AD[w,d] = np.median(AD_set[1:])
        table_med_SPJ[w,d] = np.median(SPJ_set[1:])
        table_mean_AD[w,d] = np.mean(AD_set[1:])
        table_mean_SPJ[w,d] = np.mean(SPJ_set[1:])
        table_min_AD[w,d] = np.min(AD_set[1:])
        table_min_SPJ[w,d] = np.min(SPJ_set[1:])
        table_max_AD[w,d] = np.max(AD_set[1:])
        table_max_SPJ[w,d] = np.max(SPJ_set[1:])
        table_rej_AD[w,d] = sum(x > 5.9915 for x in AD_set[1:])/(i+1)
        table_rej_SPJ[w,d] = sum(y > 5.9915 for y in SPJ_set[1:])/(i+1)
        table_rej_AD_fixed[w,d] = sum(xx>yy for xx, yy in zip(globals()[name3][1:],globals()[newname1][1:]))/(i+1)
        table_rej_SPJ_fixed[w,d] = sum(xx2>yy2 for xx2, yy2 in zip(globals()[name4][1:],globals()[newname2][1:]))/(i+1)
        table_avgband_AD[w,d] = np.mean(globals()[name1][1:])
        table_avgband_SPJ[w,d] = np.mean(globals()[name2][1:])
        table_stdband_AD[w,d] = np.std(globals()[name1][1:])
        table_stdband_SPJ[w,d] = np.std(globals()[name2][1:])
        
        table_med_NW1[w,d] = np.median(NW1_set[1:])
        table_med_NW2[w,d] = np.median(NW2_set[1:])
        table_mean_NW1[w,d] = np.mean(NW1_set[1:])
        table_mean_NW2[w,d] = np.mean(NW2_set[1:])
        table_min_NW1[w,d] = np.min(NW1_set[1:])
        table_min_NW2[w,d] = np.min(NW2_set[1:])
        table_max_NW1[w,d] = np.max(NW1_set[1:])
        table_max_NW2[w,d] = np.max(NW2_set[1:])
        table_rej_NW1[w,d] = sum(x > 5.9915 for x in NW1_set[1:])/(i+1)
        table_rej_NW2[w,d] = sum(y > 5.9915 for y in NW2_set[1:])/(i+1)
        table_rej_NW1_fixed[w,d] = sum(xx>yy for xx, yy in zip(globals()[name3_n1][1:],globals()[newname1_n1][1:]))/(i+1)
        table_rej_NW2_fixed[w,d] = sum(xx2>yy2 for xx2, yy2 in zip(globals()[name3_n2][1:],globals()[newname1_n2][1:]))/(i+1)
        table_avgband_NW1[w,d] = np.mean(globals()[name1_n1][1:])
        table_avgband_NW2[w,d] = np.mean(globals()[name1_n2][1:])
        table_stdband_NW1[w,d] = np.std(globals()[name1_n1][1:])
        table_stdband_NW2[w,d] = np.std(globals()[name1_n2][1:])
        
        
        #AD_set = [0]
        #SPJ_set = [0]


In [81]:
SPJ_band(vhat,w=2,z_a=1.96,delta=2,q=2,g=6,c=0.539)/Tnum

0.03989576452318371

In [357]:
AD_band(vhat)/500

0.06166613545238547

In [52]:
print(np.round(table_mean_AD,2))

[[123.87  79.15  71.59  69.34  67.9 ]
 [ 11.01   8.92   7.22   6.56   6.15]
 [  4.5    4.2    3.52   3.07   2.7 ]
 [  3.2    3.1    2.74   2.42   2.12]
 [  2.58   2.56   2.36   2.13   1.89]]


In [53]:
print(np.round(table_mean_SPJ,2))

[[99.87 63.58 57.29 55.38 54.17]
 [ 9.14  7.4   5.96  5.41  5.07]
 [ 3.98  3.72  3.09  2.69  2.37]
 [ 2.99  2.9   2.55  2.25  1.96]
 [ 2.54  2.52  2.32  2.09  1.85]]


In [54]:
print(np.round(table_mean_NW1,2))

[[738.07 475.17 433.7  422.3  415.55]
 [ 59.84  48.37  39.55  36.2   34.11]
 [ 19.45  18.03  15.21  13.39  11.88]
 [ 10.73  10.38   9.18   8.18   7.19]
 [  6.     6.01   5.51   4.99   4.45]]


In [55]:
print(np.round(table_mean_NW2,2))

[[56.34 34.92 30.69 29.35 28.56]
 [ 6.11  4.85  3.81  3.44  3.22]
 [ 3.36  3.16  2.59  2.19  1.92]
 [ 2.97  2.93  2.47  2.12  1.84]
 [ 2.8   2.88  2.46  2.14  1.88]]


In [56]:
print (table_rej_AD)

[[0.925 0.87  0.81  0.8   0.8  ]
 [0.555 0.465 0.35  0.31  0.285]
 [0.26  0.21  0.165 0.13  0.12 ]
 [0.16  0.14  0.1   0.09  0.085]
 [0.105 0.095 0.085 0.08  0.07 ]]


In [57]:
print(table_rej_SPJ)

[[0.895 0.85  0.795 0.78  0.77 ]
 [0.495 0.405 0.29  0.27  0.26 ]
 [0.225 0.185 0.14  0.12  0.115]
 [0.145 0.135 0.105 0.09  0.08 ]
 [0.1   0.09  0.09  0.08  0.07 ]]


In [58]:
print (table_rej_NW1)

[[0.985 0.965 0.955 0.95  0.94 ]
 [0.915 0.865 0.84  0.795 0.725]
 [0.755 0.735 0.665 0.605 0.53 ]
 [0.545 0.545 0.535 0.455 0.39 ]
 [0.355 0.395 0.35  0.31  0.25 ]]


In [59]:
print (table_rej_NW2)

[[0.795 0.71  0.67  0.655 0.63 ]
 [0.36  0.255 0.18  0.165 0.15 ]
 [0.18  0.145 0.105 0.085 0.08 ]
 [0.13  0.12  0.09  0.08  0.065]
 [0.115 0.11  0.09  0.08  0.065]]


In [60]:
print(table_rej_AD_fixed)

[[0.905 0.855 0.795 0.785 0.775]
 [0.5   0.41  0.305 0.275 0.265]
 [0.2   0.18  0.135 0.11  0.11 ]
 [0.12  0.105 0.085 0.075 0.07 ]
 [0.07  0.05  0.05  0.04  0.035]]


In [61]:
print(table_rej_SPJ_fixed)

[[0.88  0.82  0.76  0.745 0.74 ]
 [0.425 0.31  0.225 0.215 0.21 ]
 [0.155 0.13  0.09  0.085 0.075]
 [0.08  0.085 0.07  0.06  0.05 ]
 [0.07  0.04  0.045 0.04  0.035]]


In [62]:
print(table_rej_NW1_fixed)

[[0.985 0.965 0.955 0.945 0.935]
 [0.915 0.86  0.835 0.785 0.72 ]
 [0.74  0.725 0.66  0.6   0.52 ]
 [0.535 0.54  0.53  0.44  0.38 ]
 [0.345 0.385 0.34  0.295 0.23 ]]


In [63]:
print(table_rej_NW2_fixed)

[[0.745 0.62  0.585 0.565 0.56 ]
 [0.22  0.15  0.13  0.115 0.105]
 [0.09  0.085 0.07  0.055 0.04 ]
 [0.085 0.075 0.045 0.055 0.045]
 [0.075 0.055 0.035 0.035 0.03 ]]


In [65]:
print(np.round(table_avgband_AD,3))

[[0.045 0.045 0.045 0.045 0.045]
 [0.045 0.045 0.045 0.045 0.045]
 [0.045 0.045 0.045 0.045 0.045]
 [0.045 0.045 0.045 0.045 0.045]
 [0.045 0.045 0.045 0.045 0.045]]


In [66]:
print(np.round(table_avgband_SPJ,3))

[[0.058 0.058 0.058 0.058 0.058]
 [0.058 0.058 0.058 0.058 0.058]
 [0.058 0.058 0.058 0.058 0.058]
 [0.058 0.058 0.058 0.058 0.058]
 [0.058 0.058 0.058 0.058 0.058]]


In [67]:
print(np.round(table_avgband_NW1,3))

[[0.007 0.007 0.007 0.007 0.007]
 [0.007 0.007 0.007 0.007 0.007]
 [0.007 0.007 0.007 0.007 0.007]
 [0.007 0.007 0.007 0.007 0.007]
 [0.007 0.007 0.007 0.007 0.007]]


In [68]:
print(np.round(table_avgband_NW2,3))

[[0.13  0.131 0.131 0.132 0.132]
 [0.12  0.121 0.122 0.123 0.123]
 [0.097 0.098 0.099 0.101 0.101]
 [0.08  0.083 0.082 0.083 0.083]
 [0.087 0.088 0.087 0.086 0.085]]


In [69]:
print(np.round(table_stdband_AD,3))

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [70]:
print(np.round(table_stdband_SPJ,3))

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [168]:
a = band_AD44
b = band_AD33

In [73]:
table_mean_AD_D = np.copy(table_mean_AD)
table_mean_SPJ_D = np.copy(table_mean_SPJ)
table_mean_NW1_D = np.copy(table_mean_NW1)
table_mean_NW2_D = np.copy(table_mean_NW2)

table_rej_AD_D = np.copy(table_rej_AD)
table_rej_SPJ_D = np.copy(table_rej_SPJ)
table_rej_NW1_D = np.copy(table_rej_NW1)
table_rej_NW2_D = np.copy(table_rej_NW2)

table_rej_AD_fixed_D = np.copy(table_rej_AD_fixed)
table_rej_SPJ_fixed_D = np.copy(table_rej_SPJ_fixed)
table_rej_NW1_fixed_D = np.copy(table_rej_NW1_fixed)
table_rej_Nw2_fixed_D = np.copy(table_rej_NW2_fixed)

table_avgband_AD_D = np.copy(table_avgband_AD)
table_avgband_SPJ_D = np.copy(table_avgband_SPJ)
table_avgband_NW1_D = np.copy(table_avgband_NW1)
table_avgband_NW2_D = np.copy(table_avgband_NW2)

table_stdband_AD_D = np.copy(table_stdband_AD)
table_stdband_SPJ_D = np.copy(table_stdband_SPJ)
table_stdband_NW1_D = np.copy(table_stdband_NW1)
table_stdband_NW2_D = np.copy(table_stdband_NW2)

In [108]:
c1 = 0.26880
c2 = 0.98804
c3 = -0.04904
c4 = 5.13641
c5 = -2.37697
c6 = 0.15189
c7 = -2.17925
c8 = 3.06497
c9 = 0.22194
ku = np.linspace(0,6,5) + 0
ku = np.insert(ku, 1, 0.5)
ku = np.delete(ku, 4)
kx = np.linspace(0,6,5) + 0
kx = np.insert(kx, 1, 0.5)
kx = np.delete(kx, 4)
kx[2] = kx[2] + 0.0718
ku[2] = ku[2] + 0.0718
table_mean_AD = np.zeros((5,5))
table_mean_SPJ = np.zeros((5,5))
table_med_AD = np.zeros((5,5))
table_med_SPJ = np.zeros((5,5))
table_min_AD = np.zeros((5,5))
table_min_SPJ = np.zeros((5,5))
table_max_AD = np.zeros((5,5))
table_max_SPJ = np.zeros((5,5))

table_rej_AD = np.zeros((5,5))
table_rej_SPJ = np.zeros((5,5))

table_rej_AD_fixed = np.zeros((5,5))
table_rej_SPJ_fixed = np.zeros((5,5))

table_avgband_AD = np.zeros((5,5))
table_avgband_SPJ = np.zeros((5,5))

table_stdband_AD = np.zeros((5,5))
table_stdband_SPJ = np.zeros((5,5))

table_mean_NW1 = np.zeros((5,5))
table_mean_NW2 = np.zeros((5,5))
table_med_NW1 = np.zeros((5,5))
table_med_NW2 = np.zeros((5,5))
table_min_NW1 = np.zeros((5,5))
table_min_NW2 = np.zeros((5,5))
table_max_NW1 = np.zeros((5,5))
table_max_NW2 = np.zeros((5,5))

table_rej_NW1 = np.zeros((5,5))
table_rej_NW2 = np.zeros((5,5))

table_rej_NW1_fixed = np.zeros((5,5))
table_rej_NW2_fixed = np.zeros((5,5))

table_avgband_NW1 = np.zeros((5,5))
table_avgband_NW2 = np.zeros((5,5))

table_stdband_NW1 = np.zeros((5,5))
table_stdband_NW2 = np.zeros((5,5))


AD_set = [0]
SPJ_set = [0]
NW1_set = [0]
NW2_set = [0]

for w in range(len(ku)):
    for d in range(len(kx)):
        AD_set = [0]
        SPJ_set = [0]
        NW1_set = [0]
        NW2_set = [0]
        
        name1 = "band_AD"+str(w)+str(d)
        name2 = "band_SPJ"+str(w)+str(d)
        name1_n1 = "band_NW1"+str(w)+str(d)
        name1_n2 = "band_NW2"+str(w)+str(d)
        globals()[name1] = [0]
        globals()[name2] = [0]
        globals()[name1_n1] = [0]
        globals()[name1_n2] = [0]
        
        newname1 = "fixed_AD"+str(w)+str(d)
        newname2 = "fixed_SPJ"+str(w)+str(d)
        newname1_n1 = "fixed_NW1"+str(w)+str(d)
        newname1_n2 = "fixed_NW2"+str(w)+str(d)
        globals()[newname1] = [0]
        globals()[newname2] = [0]
        globals()[newname1_n1] = [0]
        globals()[newname1_n2] = [0]
        
        name3 = "stat_AD"+str(w)+str(d)
        name4 = "stat_SPJ"+str(w)+str(d)
        name3_n1 = "stat_NW1"+str(w)+str(d)
        name3_n2 = "stat_NW2"+str(w)+str(d)
        globals()[name3] = [0]
        globals()[name4] = [0]
        globals()[name3_n1] = [0]
        globals()[name3_n2] = [0]
        
        for i in range(200):
            np.random.seed(i)
            u = OU_p(0,30,30*252,ku[w],0,0.0097)
            x = OU_p(0,30,30*252,kx[d],0,0.0998)
            
            #u = OU_AR(0,30,50,ku[w],0,0.0097)
            #x = OU_AR(0,30,50,kx[d],0,0.0998)
            
            u = np.copy(u[::2]) #quaterly
            x = np.copy(x[::2])
            Tnum = x.shape[0]
            
            b0 = 0
            b1 = 1
            y = b0 + b1*x + u
            nu = len(x)
            cons = np.ones((nu,1))
            X = np.hstack((cons,x))
            coeffs = np.linalg.inv(X.transpose().dot(X)).dot(X.transpose()).dot(y)

            X_hat = np.dot(X,coeffs)
            uhat = y - X_hat
            vhat = np.copy(uhat) #G-test
            #vhat = np.copy(X*uhat) #H-test

            R = np.array([[1, 0], [0, 1]])
            r = np.array([[0],[1]])
            
            #####################################################################
            
            vhat2 = np.copy(vhat)
            AD1 = G_test(R,r,coeffs,X,vhat2,c=1,p=4/25,q=2,band="AD")
            #AD1 = H_test(R,r,coeffs,X,vhat,c=1,p=4/25,q=2,band="AD")
            AD_set.append(AD1)
            globals()[name3].append(AD1)
            vhat2 = np.copy(vhat)
            AD_b = AD_band(vhat2)
            globals()[name1].append(AD_b/Tnum)
            
            AD_fixed_cv = 5.9915 + c1*(AD_b/Tnum*5.9915) + c2*((AD_b/Tnum)*(5.9915**2)) + c3*((AD_b/Tnum)*(5.9915**3)) + c4*(((AD_b/Tnum)**2)*5.9915) + c5*(((AD_b/Tnum)**2)*(5.9915**2)) + c6*(((AD_b/Tnum)**2)*(5.9915**3)) + c7*(((AD_b/Tnum)**3)*(5.9915**1)) + c8*(((AD_b/Tnum)**3)*(5.9915**2)) + c9*(((AD_b/Tnum)**3)*(5.9915**3))
            globals()[newname1].append(AD_fixed_cv)
            
            #####################################################################
            vhat3 = np.copy(vhat)
            SPJ1 = G_test(R,r,coeffs,X,vhat3,c=8.5*((1/100)**(21/25)),p=21/25,q=2,band="SPJ")
            #SPJ1 = H_test(R,r,coeffs,X,vhat,c=8.5*((1/100)**(21/25)),p=21/25,q=2,band="SPJ")
            SPJ_set.append(SPJ1)
            globals()[name4].append(SPJ1)
            vhat3 = np.copy(vhat)
            SPJ_b = SPJ_band(vhat3,w=5,z_a=1.96,delta=2,q=2,g=6,c=0.539) #w is tuning parameter
            globals()[name2].append(SPJ_b/Tnum)
            
            SPJ_fixed_cv = 5.9915 + c1*(SPJ_b/Tnum*5.9915) + c2*((SPJ_b/Tnum)*(5.9915**2)) + c3*((SPJ_b/Tnum)*(5.9915**3)) + c4*(((SPJ_b/Tnum)**2)*5.9915) + c5*(((SPJ_b/Tnum)**2)*(5.9915**2)) + c6*(((SPJ_b/Tnum)**2)*(5.9915**3)) + c7*(((SPJ_b/Tnum)**3)*(5.9915**1)) + c8*(((SPJ_b/Tnum)**3)*(5.9915**2)) + c9*(((SPJ_b/Tnum)**3)*(5.9915**3))
            globals()[newname2].append(SPJ_fixed_cv)
            
            #####################################################################
            vhat3 = np.copy(vhat)
            NW1 = G_test(R,r,coeffs,X,vhat3,c=4*((1/100)**(4/25)),p=4/25,q=2,band="NW")
            #NW1 = H_test(R,r,coeffs,X,vhat3,c=4*((1/100)**(4/25)),p=4/25,q=2,band="NW")
            NW1_set.append(NW1)
            globals()[name3_n1].append(NW1)
            vhat3 = np.copy(vhat)
            NW1_b = NW_band(vhat3,c=4*((1/100)**(4/25)),p=4/25,q=2)
            globals()[name1_n1].append(NW1_b/Tnum)
            
            NW1_fixed_cv = 5.9915 + c1*(NW1_b/Tnum*5.9915) + c2*((NW1_b/Tnum)*(5.9915**2)) + c3*((NW1_b/Tnum)*(5.9915**3)) + c4*(((NW1_b/Tnum)**2)*5.9915) + c5*(((NW1_b/Tnum)**2)*(5.9915**2)) + c6*(((NW1_b/Tnum)**2)*(5.9915**3)) + c7*(((NW1_b/Tnum)**3)*(5.9915**1)) + c8*(((NW1_b/Tnum)**3)*(5.9915**2)) + c9*(((NW1_b/Tnum)**3)*(5.9915**3))
            globals()[newname1_n1].append(NW1_fixed_cv)
            
            #####################################################################
            vhat3 = np.copy(vhat)
            NW2 = G_test(R,r,coeffs,X,vhat3,c=8.5*((1/100)**(21/25)),p=21/25,q=2,band="NW")
            #NW2 = H_test(R,r,coeffs,X,vhat3,c=4*((1/100)**(4/25)),p=4/25,q=2,band="NW")
            NW2_set.append(NW2)
            globals()[name3_n2].append(NW2)
            vhat3 = np.copy(vhat)
            NW2_b = NW_band(vhat3,c=8.5*((1/100)**(21/25)),p=21/25,q=2)
            globals()[name1_n2].append(NW2_b/Tnum)
            
            NW2_fixed_cv = 5.9915 + c1*(NW2_b/Tnum*5.9915) + c2*((NW2_b/Tnum)*(5.9915**2)) + c3*((NW2_b/Tnum)*(5.9915**3)) + c4*(((NW2_b/Tnum)**2)*5.9915) + c5*(((NW2_b/Tnum)**2)*(5.9915**2)) + c6*(((NW2_b/Tnum)**2)*(5.9915**3)) + c7*(((NW2_b/Tnum)**3)*(5.9915**1)) + c8*(((NW2_b/Tnum)**3)*(5.9915**2)) + c9*(((NW2_b/Tnum)**3)*(5.9915**3))
            globals()[newname1_n2].append(NW2_fixed_cv)
            
        table_med_AD[w,d] = np.median(AD_set[1:])
        table_med_SPJ[w,d] = np.median(SPJ_set[1:])
        table_mean_AD[w,d] = np.mean(AD_set[1:])
        table_mean_SPJ[w,d] = np.mean(SPJ_set[1:])
        table_min_AD[w,d] = np.min(AD_set[1:])
        table_min_SPJ[w,d] = np.min(SPJ_set[1:])
        table_max_AD[w,d] = np.max(AD_set[1:])
        table_max_SPJ[w,d] = np.max(SPJ_set[1:])
        table_rej_AD[w,d] = sum(x > 5.9915 for x in AD_set[1:])/(i+1)
        table_rej_SPJ[w,d] = sum(y > 5.9915 for y in SPJ_set[1:])/(i+1)
        table_rej_AD_fixed[w,d] = sum(xx>yy for xx, yy in zip(globals()[name3][1:],globals()[newname1][1:]))/(i+1)
        table_rej_SPJ_fixed[w,d] = sum(xx2>yy2 for xx2, yy2 in zip(globals()[name4][1:],globals()[newname2][1:]))/(i+1)
        table_avgband_AD[w,d] = np.mean(globals()[name1][1:])
        table_avgband_SPJ[w,d] = np.mean(globals()[name2][1:])
        table_stdband_AD[w,d] = np.std(globals()[name1][1:])
        table_stdband_SPJ[w,d] = np.std(globals()[name2][1:])
        
        table_med_NW1[w,d] = np.median(NW1_set[1:])
        table_med_NW2[w,d] = np.median(NW2_set[1:])
        table_mean_NW1[w,d] = np.mean(NW1_set[1:])
        table_mean_NW2[w,d] = np.mean(NW2_set[1:])
        table_min_NW1[w,d] = np.min(NW1_set[1:])
        table_min_NW2[w,d] = np.min(NW2_set[1:])
        table_max_NW1[w,d] = np.max(NW1_set[1:])
        table_max_NW2[w,d] = np.max(NW2_set[1:])
        table_rej_NW1[w,d] = sum(x > 5.9915 for x in NW1_set[1:])/(i+1)
        table_rej_NW2[w,d] = sum(y > 5.9915 for y in NW2_set[1:])/(i+1)
        table_rej_NW1_fixed[w,d] = sum(xx>yy for xx, yy in zip(globals()[name3_n1][1:],globals()[newname1_n1][1:]))/(i+1)
        table_rej_NW2_fixed[w,d] = sum(xx2>yy2 for xx2, yy2 in zip(globals()[name3_n2][1:],globals()[newname1_n2][1:]))/(i+1)
        table_avgband_NW1[w,d] = np.mean(globals()[name1_n1][1:])
        table_avgband_NW2[w,d] = np.mean(globals()[name1_n2][1:])
        table_stdband_NW1[w,d] = np.std(globals()[name1_n1][1:])
        table_stdband_NW2[w,d] = np.std(globals()[name1_n2][1:])
        
        
        #AD_set = [0]
        #SPJ_set = [0]


In [116]:
table_mean_AD_2D = np.copy(table_mean_AD)
table_mean_SPJ_2D = np.copy(table_mean_SPJ)
table_mean_NW1_2D = np.copy(table_mean_NW1)
table_mean_NW2_2D = np.copy(table_mean_NW2)

table_rej_AD_2D = np.copy(table_rej_AD)
table_rej_SPJ_2D = np.copy(table_rej_SPJ)
table_rej_NW1_2D = np.copy(table_rej_NW1)
table_rej_NW2_2D = np.copy(table_rej_NW2)

table_rej_AD_fixed_2D = np.copy(table_rej_AD_fixed)
table_rej_SPJ_fixed_2D = np.copy(table_rej_SPJ_fixed)
table_rej_NW1_fixed_2D = np.copy(table_rej_NW1_fixed)
table_rej_NW2_fixed_2D = np.copy(table_rej_NW2_fixed)

table_avgband_AD_2D = np.copy(table_avgband_AD)
table_avgband_SPJ_2D = np.copy(table_avgband_SPJ)
table_avgband_NW1_2D = np.copy(table_avgband_NW1)
table_avgband_NW2_2D = np.copy(table_avgband_NW2)

table_stdband_AD_2D = np.copy(table_stdband_AD)
table_stdband_SPJ_2D = np.copy(table_stdband_SPJ)
table_stdband_NW1_2D = np.copy(table_stdband_NW1)
table_stdband_NW2_2D = np.copy(table_stdband_NW2)

In [110]:
print(np.round(table_mean_AD_2D,2))
print(np.round(table_mean_SPJ_2D,2))
print(np.round(table_mean_NW1_2D,2))
print(np.round(table_mean_NW2_2D,2))

[[78.8  49.85 44.64 43.04 42.03]
 [ 7.54  6.08  4.86  4.41  4.12]
 [ 3.56  3.34  2.75  2.38  2.09]
 [ 2.85  2.77  2.42  2.12  1.84]
 [ 2.55  2.53  2.33  2.11  1.87]]
[[69.89 44.02 39.26 37.79 36.87]
 [ 6.89  5.54  4.41  3.99  3.73]
 [ 3.42  3.21  2.62  2.26  1.99]
 [ 2.82  2.76  2.38  2.08  1.8 ]
 [ 2.54  2.52  2.31  2.08  1.84]]
[[430.69 276.99 252.57 245.8  241.72]
 [ 35.41  28.65  23.41  21.4   20.15]
 [ 11.94  11.08   9.35   8.23   7.29]
 [  6.89   6.67   5.91   5.26   4.63]
 [  4.17   4.18   3.84   3.48   3.11]]
[[53.76 33.17 29.05 27.76 26.99]
 [ 5.98  4.72  3.7   3.34  3.12]
 [ 3.4   3.21  2.56  2.19  1.95]
 [ 3.    3.12  2.56  2.13  1.89]
 [ 2.86  7.28  2.71  2.22  1.97]]


In [112]:
print((table_rej_AD_2D))
print((table_rej_SPJ_2D))
print(table_rej_NW1_2D)
print(table_rej_NW2_2D)

[[0.875 0.81  0.75  0.74  0.735]
 [0.425 0.33  0.235 0.215 0.21 ]
 [0.18  0.15  0.105 0.09  0.09 ]
 [0.125 0.125 0.095 0.085 0.075]
 [0.105 0.09  0.085 0.08  0.07 ]]
[[0.845 0.78  0.725 0.715 0.705]
 [0.39  0.285 0.21  0.2   0.195]
 [0.175 0.145 0.1   0.09  0.085]
 [0.125 0.115 0.09  0.08  0.065]
 [0.1   0.1   0.08  0.075 0.07 ]]
[[0.98  0.935 0.935 0.92  0.91 ]
 [0.825 0.78  0.715 0.655 0.6  ]
 [0.585 0.605 0.52  0.415 0.37 ]
 [0.395 0.425 0.34  0.295 0.23 ]
 [0.24  0.25  0.2   0.17  0.135]]
[[0.79  0.7   0.665 0.63  0.625]
 [0.345 0.24  0.165 0.16  0.15 ]
 [0.18  0.15  0.105 0.095 0.09 ]
 [0.145 0.14  0.1   0.085 0.085]
 [0.135 0.13  0.09  0.085 0.08 ]]


In [117]:
print((table_rej_AD_fixed_2D))
print((table_rej_SPJ_fixed_2D))
print(table_rej_NW1_fixed_2D)
print(table_rej_NW2_fixed_2D)

[[0.8   0.755 0.695 0.685 0.675]
 [0.335 0.235 0.165 0.16  0.15 ]
 [0.11  0.1   0.065 0.065 0.05 ]
 [0.07  0.06  0.055 0.045 0.025]
 [0.07  0.045 0.045 0.04  0.035]]
[[0.785 0.7   0.665 0.635 0.63 ]
 [0.28  0.195 0.155 0.14  0.135]
 [0.1   0.09  0.06  0.055 0.045]
 [0.065 0.055 0.035 0.035 0.025]
 [0.065 0.045 0.045 0.035 0.03 ]]
[[0.98  0.935 0.93  0.915 0.9  ]
 [0.82  0.775 0.695 0.64  0.59 ]
 [0.565 0.56  0.495 0.41  0.355]
 [0.38  0.4   0.325 0.285 0.225]
 [0.22  0.24  0.19  0.14  0.13 ]]
[[0.725 0.585 0.55  0.535 0.53 ]
 [0.215 0.155 0.115 0.11  0.105]
 [0.09  0.08  0.05  0.04  0.035]
 [0.05  0.065 0.055 0.04  0.03 ]
 [0.06  0.045 0.04  0.035 0.02 ]]


In [121]:
print(np.round(table_avgband_AD_2D,3))
print(np.round(table_avgband_SPJ_2D,3))
print(np.round(table_avgband_NW1_2D,3))
print(np.round(table_avgband_NW2_2D,3))

[[0.079 0.079 0.079 0.079 0.079]
 [0.079 0.079 0.079 0.079 0.079]
 [0.079 0.079 0.079 0.079 0.079]
 [0.078 0.078 0.078 0.078 0.078]
 [0.054 0.053 0.054 0.054 0.054]]
[[0.093 0.093 0.093 0.093 0.093]
 [0.093 0.093 0.093 0.093 0.093]
 [0.093 0.093 0.093 0.093 0.093]
 [0.092 0.092 0.092 0.092 0.092]
 [0.067 0.067 0.067 0.067 0.067]]
[[0.012 0.012 0.012 0.012 0.012]
 [0.012 0.012 0.012 0.012 0.012]
 [0.012 0.012 0.012 0.012 0.012]
 [0.012 0.012 0.012 0.012 0.012]
 [0.011 0.011 0.011 0.011 0.011]]
[[0.14  0.142 0.143 0.143 0.144]
 [0.128 0.13  0.131 0.132 0.132]
 [0.101 0.103 0.104 0.105 0.105]
 [0.089 0.091 0.09  0.09  0.09 ]
 [0.097 0.102 0.098 0.095 0.094]]


In [163]:
c1 = 0.26880
c2 = 0.98804
c3 = -0.04904
c4 = 5.13641
c5 = -2.37697
c6 = 0.15189
c7 = -2.17925
c8 = 3.06497
c9 = 0.22194
ku = np.linspace(0,6,5) + 0
ku = np.insert(ku, 1, 0.5)
ku = np.delete(ku, 4)
kx = np.linspace(0,6,5) + 0
kx = np.insert(kx, 1, 0.5)
kx = np.delete(kx, 4)
kx[2] = kx[2] + 0.0718
ku[2] = ku[2] + 0.0718
table_mean_AD = np.zeros((5,5))
table_mean_SPJ = np.zeros((5,5))
table_med_AD = np.zeros((5,5))
table_med_SPJ = np.zeros((5,5))
table_min_AD = np.zeros((5,5))
table_min_SPJ = np.zeros((5,5))
table_max_AD = np.zeros((5,5))
table_max_SPJ = np.zeros((5,5))

table_rej_AD = np.zeros((5,5))
table_rej_SPJ = np.zeros((5,5))

table_rej_AD_fixed = np.zeros((5,5))
table_rej_SPJ_fixed = np.zeros((5,5))

table_avgband_AD = np.zeros((5,5))
table_avgband_SPJ = np.zeros((5,5))

table_stdband_AD = np.zeros((5,5))
table_stdband_SPJ = np.zeros((5,5))

table_mean_NW1 = np.zeros((5,5))
table_mean_NW2 = np.zeros((5,5))
table_med_NW1 = np.zeros((5,5))
table_med_NW2 = np.zeros((5,5))
table_min_NW1 = np.zeros((5,5))
table_min_NW2 = np.zeros((5,5))
table_max_NW1 = np.zeros((5,5))
table_max_NW2 = np.zeros((5,5))

table_rej_NW1 = np.zeros((5,5))
table_rej_NW2 = np.zeros((5,5))

table_rej_NW1_fixed = np.zeros((5,5))
table_rej_NW2_fixed = np.zeros((5,5))

table_avgband_NW1 = np.zeros((5,5))
table_avgband_NW2 = np.zeros((5,5))

table_stdband_NW1 = np.zeros((5,5))
table_stdband_NW2 = np.zeros((5,5))


AD_set = [0]
SPJ_set = [0]
NW1_set = [0]
NW2_set = [0]

for w in range(len(ku)):
    for d in range(len(kx)):
        AD_set = [0]
        SPJ_set = [0]
        NW1_set = [0]
        NW2_set = [0]
        
        name1 = "band_AD"+str(w)+str(d)
        name2 = "band_SPJ"+str(w)+str(d)
        name1_n1 = "band_NW1"+str(w)+str(d)
        name1_n2 = "band_NW2"+str(w)+str(d)
        globals()[name1] = [0]
        globals()[name2] = [0]
        globals()[name1_n1] = [0]
        globals()[name1_n2] = [0]
        
        newname1 = "fixed_AD"+str(w)+str(d)
        newname2 = "fixed_SPJ"+str(w)+str(d)
        newname1_n1 = "fixed_NW1"+str(w)+str(d)
        newname1_n2 = "fixed_NW2"+str(w)+str(d)
        globals()[newname1] = [0]
        globals()[newname2] = [0]
        globals()[newname1_n1] = [0]
        globals()[newname1_n2] = [0]
        
        name3 = "stat_AD"+str(w)+str(d)
        name4 = "stat_SPJ"+str(w)+str(d)
        name3_n1 = "stat_NW1"+str(w)+str(d)
        name3_n2 = "stat_NW2"+str(w)+str(d)
        globals()[name3] = [0]
        globals()[name4] = [0]
        globals()[name3_n1] = [0]
        globals()[name3_n2] = [0]
        
        for i in range(200):
            np.random.seed(i)
            u = OU_p(0,30,30*252,ku[w],0,0.0097)
            x = OU_p(0,30,30*252,kx[d],0,0.0998)
            
            #u = OU_AR(0,30,50,ku[w],0,0.0097)
            #x = OU_AR(0,30,50,kx[d],0,0.0998)
            
            u = np.copy(u[::5]) #quaterly
            x = np.copy(x[::5])
            Tnum = x.shape[0]
            
            b0 = 0
            b1 = 1
            y = b0 + b1*x + u
            nu = len(x)
            cons = np.ones((nu,1))
            X = np.hstack((cons,x))
            coeffs = np.linalg.inv(X.transpose().dot(X)).dot(X.transpose()).dot(y)

            X_hat = np.dot(X,coeffs)
            uhat = y - X_hat
            vhat = np.copy(uhat) #G-test
            #vhat = np.copy(X*uhat) #H-test

            R = np.array([[1, 0], [0, 1]])
            r = np.array([[0],[1]])
            
            #####################################################################
            
            vhat2 = np.copy(vhat)
            AD1 = G_test(R,r,coeffs,X,vhat2,c=1,p=4/25,q=2,band="AD")
            #AD1 = H_test(R,r,coeffs,X,vhat,c=1,p=4/25,q=2,band="AD")
            AD_set.append(AD1)
            globals()[name3].append(AD1)
            vhat2 = np.copy(vhat)
            AD_b = AD_band(vhat2)
            globals()[name1].append(AD_b/Tnum)
            
            AD_fixed_cv = 5.9915 + c1*(AD_b/Tnum*5.9915) + c2*((AD_b/Tnum)*(5.9915**2)) + c3*((AD_b/Tnum)*(5.9915**3)) + c4*(((AD_b/Tnum)**2)*5.9915) + c5*(((AD_b/Tnum)**2)*(5.9915**2)) + c6*(((AD_b/Tnum)**2)*(5.9915**3)) + c7*(((AD_b/Tnum)**3)*(5.9915**1)) + c8*(((AD_b/Tnum)**3)*(5.9915**2)) + c9*(((AD_b/Tnum)**3)*(5.9915**3))
            globals()[newname1].append(AD_fixed_cv)
            
            #####################################################################
            vhat3 = np.copy(vhat)
            SPJ1 = G_test(R,r,coeffs,X,vhat3,c=8.5*((1/100)**(21/25)),p=21/25,q=2,band="SPJ")
            #SPJ1 = H_test(R,r,coeffs,X,vhat,c=8.5*((1/100)**(21/25)),p=21/25,q=2,band="SPJ")
            SPJ_set.append(SPJ1)
            globals()[name4].append(SPJ1)
            vhat3 = np.copy(vhat)
            SPJ_b = SPJ_band(vhat3,w=5,z_a=1.96,delta=2,q=2,g=6,c=0.539) #w is tuning parameter
            globals()[name2].append(SPJ_b/Tnum)
            
            SPJ_fixed_cv = 5.9915 + c1*(SPJ_b/Tnum*5.9915) + c2*((SPJ_b/Tnum)*(5.9915**2)) + c3*((SPJ_b/Tnum)*(5.9915**3)) + c4*(((SPJ_b/Tnum)**2)*5.9915) + c5*(((SPJ_b/Tnum)**2)*(5.9915**2)) + c6*(((SPJ_b/Tnum)**2)*(5.9915**3)) + c7*(((SPJ_b/Tnum)**3)*(5.9915**1)) + c8*(((SPJ_b/Tnum)**3)*(5.9915**2)) + c9*(((SPJ_b/Tnum)**3)*(5.9915**3))
            globals()[newname2].append(SPJ_fixed_cv)
            
            #####################################################################
            vhat3 = np.copy(vhat)
            NW1 = G_test(R,r,coeffs,X,vhat3,c=4*((1/100)**(4/25)),p=4/25,q=2,band="NW")
            #NW1 = H_test(R,r,coeffs,X,vhat3,c=4*((1/100)**(4/25)),p=4/25,q=2,band="NW")
            NW1_set.append(NW1)
            globals()[name3_n1].append(NW1)
            vhat3 = np.copy(vhat)
            NW1_b = NW_band(vhat3,c=4*((1/100)**(4/25)),p=4/25,q=2)
            globals()[name1_n1].append(NW1_b/Tnum)
            
            NW1_fixed_cv = 5.9915 + c1*(NW1_b/Tnum*5.9915) + c2*((NW1_b/Tnum)*(5.9915**2)) + c3*((NW1_b/Tnum)*(5.9915**3)) + c4*(((NW1_b/Tnum)**2)*5.9915) + c5*(((NW1_b/Tnum)**2)*(5.9915**2)) + c6*(((NW1_b/Tnum)**2)*(5.9915**3)) + c7*(((NW1_b/Tnum)**3)*(5.9915**1)) + c8*(((NW1_b/Tnum)**3)*(5.9915**2)) + c9*(((NW1_b/Tnum)**3)*(5.9915**3))
            globals()[newname1_n1].append(NW1_fixed_cv)
            
            #####################################################################
            vhat3 = np.copy(vhat)
            NW2 = G_test(R,r,coeffs,X,vhat3,c=8.5*((1/100)**(21/25)),p=21/25,q=2,band="NW")
            #NW2 = H_test(R,r,coeffs,X,vhat3,c=4*((1/100)**(4/25)),p=4/25,q=2,band="NW")
            NW2_set.append(NW2)
            globals()[name3_n2].append(NW2)
            vhat3 = np.copy(vhat)
            NW2_b = NW_band(vhat3,c=8.5*((1/100)**(21/25)),p=21/25,q=2)
            globals()[name1_n2].append(NW2_b/Tnum)
            
            NW2_fixed_cv = 5.9915 + c1*(NW2_b/Tnum*5.9915) + c2*((NW2_b/Tnum)*(5.9915**2)) + c3*((NW2_b/Tnum)*(5.9915**3)) + c4*(((NW2_b/Tnum)**2)*5.9915) + c5*(((NW2_b/Tnum)**2)*(5.9915**2)) + c6*(((NW2_b/Tnum)**2)*(5.9915**3)) + c7*(((NW2_b/Tnum)**3)*(5.9915**1)) + c8*(((NW2_b/Tnum)**3)*(5.9915**2)) + c9*(((NW2_b/Tnum)**3)*(5.9915**3))
            globals()[newname1_n2].append(NW2_fixed_cv)
            
        table_med_AD[w,d] = np.median(AD_set[1:])
        table_med_SPJ[w,d] = np.median(SPJ_set[1:])
        table_mean_AD[w,d] = np.mean(AD_set[1:])
        table_mean_SPJ[w,d] = np.mean(SPJ_set[1:])
        table_min_AD[w,d] = np.min(AD_set[1:])
        table_min_SPJ[w,d] = np.min(SPJ_set[1:])
        table_max_AD[w,d] = np.max(AD_set[1:])
        table_max_SPJ[w,d] = np.max(SPJ_set[1:])
        table_rej_AD[w,d] = sum(x > 5.9915 for x in AD_set[1:])/(i+1)
        table_rej_SPJ[w,d] = sum(y > 5.9915 for y in SPJ_set[1:])/(i+1)
        table_rej_AD_fixed[w,d] = sum(xx>yy for xx, yy in zip(globals()[name3][1:],globals()[newname1][1:]))/(i+1)
        table_rej_SPJ_fixed[w,d] = sum(xx2>yy2 for xx2, yy2 in zip(globals()[name4][1:],globals()[newname2][1:]))/(i+1)
        table_avgband_AD[w,d] = np.mean(globals()[name1][1:])
        table_avgband_SPJ[w,d] = np.mean(globals()[name2][1:])
        table_stdband_AD[w,d] = np.std(globals()[name1][1:])
        table_stdband_SPJ[w,d] = np.std(globals()[name2][1:])
        
        table_med_NW1[w,d] = np.median(NW1_set[1:])
        table_med_NW2[w,d] = np.median(NW2_set[1:])
        table_mean_NW1[w,d] = np.mean(NW1_set[1:])
        table_mean_NW2[w,d] = np.mean(NW2_set[1:])
        table_min_NW1[w,d] = np.min(NW1_set[1:])
        table_min_NW2[w,d] = np.min(NW2_set[1:])
        table_max_NW1[w,d] = np.max(NW1_set[1:])
        table_max_NW2[w,d] = np.max(NW2_set[1:])
        table_rej_NW1[w,d] = sum(x > 5.9915 for x in NW1_set[1:])/(i+1)
        table_rej_NW2[w,d] = sum(y > 5.9915 for y in NW2_set[1:])/(i+1)
        table_rej_NW1_fixed[w,d] = sum(xx>yy for xx, yy in zip(globals()[name3_n1][1:],globals()[newname1_n1][1:]))/(i+1)
        table_rej_NW2_fixed[w,d] = sum(xx2>yy2 for xx2, yy2 in zip(globals()[name3_n2][1:],globals()[newname1_n2][1:]))/(i+1)
        table_avgband_NW1[w,d] = np.mean(globals()[name1_n1][1:])
        table_avgband_NW2[w,d] = np.mean(globals()[name1_n2][1:])
        table_stdband_NW1[w,d] = np.std(globals()[name1_n1][1:])
        table_stdband_NW2[w,d] = np.std(globals()[name1_n2][1:])
        
        
        #AD_set = [0]
        #SPJ_set = [0]


In [164]:
table_mean_AD_5D = np.copy(table_mean_AD)
table_mean_SPJ_5D = np.copy(table_mean_SPJ)
table_mean_NW1_5D = np.copy(table_mean_NW1)
table_mean_NW2_5D = np.copy(table_mean_NW2)

table_rej_AD_5D = np.copy(table_rej_AD)
table_rej_SPJ_5D = np.copy(table_rej_SPJ)
table_rej_NW1_5D = np.copy(table_rej_NW1)
table_rej_NW2_5D = np.copy(table_rej_NW2)

table_rej_AD_fixed_5D = np.copy(table_rej_AD_fixed)
table_rej_SPJ_fixed_5D = np.copy(table_rej_SPJ_fixed)
table_rej_NW1_fixed_5D = np.copy(table_rej_NW1_fixed)
table_rej_NW2_fixed_5D = np.copy(table_rej_NW2_fixed)

table_avgband_AD_5D = np.copy(table_avgband_AD)
table_avgband_SPJ_5D = np.copy(table_avgband_SPJ)
table_avgband_NW1_5D = np.copy(table_avgband_NW1)
table_avgband_NW2_5D = np.copy(table_avgband_NW2)

table_stdband_AD_5D = np.copy(table_stdband_AD)
table_stdband_SPJ_5D = np.copy(table_stdband_SPJ)
table_stdband_NW1_5D = np.copy(table_stdband_NW1)
table_stdband_NW2_5D = np.copy(table_stdband_NW2)

In [179]:
print (table_rej_AD_5D)

[[0.775 0.68  0.65  0.625 0.61 ]
 [0.33  0.215 0.165 0.165 0.145]
 [0.18  0.145 0.095 0.09  0.08 ]
 [0.125 0.125 0.09  0.08  0.065]
 [0.1   0.095 0.085 0.08  0.065]]


In [180]:
c1 = 0.26880
c2 = 0.98804
c3 = -0.04904
c4 = 5.13641
c5 = -2.37697
c6 = 0.15189
c7 = -2.17925
c8 = 3.06497
c9 = 0.22194
ku = np.linspace(0,6,5) + 0
ku = np.insert(ku, 1, 0.5)
ku = np.delete(ku, 4)
kx = np.linspace(0,6,5) + 0
kx = np.insert(kx, 1, 0.5)
kx = np.delete(kx, 4)
kx[2] = kx[2] + 0.0718
ku[2] = ku[2] + 0.0718
table_mean_AD = np.zeros((5,5))
table_mean_SPJ = np.zeros((5,5))
table_med_AD = np.zeros((5,5))
table_med_SPJ = np.zeros((5,5))
table_min_AD = np.zeros((5,5))
table_min_SPJ = np.zeros((5,5))
table_max_AD = np.zeros((5,5))
table_max_SPJ = np.zeros((5,5))

table_rej_AD = np.zeros((5,5))
table_rej_SPJ = np.zeros((5,5))

table_rej_AD_fixed = np.zeros((5,5))
table_rej_SPJ_fixed = np.zeros((5,5))

table_avgband_AD = np.zeros((5,5))
table_avgband_SPJ = np.zeros((5,5))

table_stdband_AD = np.zeros((5,5))
table_stdband_SPJ = np.zeros((5,5))

table_mean_NW1 = np.zeros((5,5))
table_mean_NW2 = np.zeros((5,5))
table_med_NW1 = np.zeros((5,5))
table_med_NW2 = np.zeros((5,5))
table_min_NW1 = np.zeros((5,5))
table_min_NW2 = np.zeros((5,5))
table_max_NW1 = np.zeros((5,5))
table_max_NW2 = np.zeros((5,5))

table_rej_NW1 = np.zeros((5,5))
table_rej_NW2 = np.zeros((5,5))

table_rej_NW1_fixed = np.zeros((5,5))
table_rej_NW2_fixed = np.zeros((5,5))

table_avgband_NW1 = np.zeros((5,5))
table_avgband_NW2 = np.zeros((5,5))

table_stdband_NW1 = np.zeros((5,5))
table_stdband_NW2 = np.zeros((5,5))


AD_set = [0]
SPJ_set = [0]
NW1_set = [0]
NW2_set = [0]

for w in range(len(ku)):
    for d in range(len(kx)):
        AD_set = [0]
        SPJ_set = [0]
        NW1_set = [0]
        NW2_set = [0]
        
        name1 = "band_AD"+str(w)+str(d)
        name2 = "band_SPJ"+str(w)+str(d)
        name1_n1 = "band_NW1"+str(w)+str(d)
        name1_n2 = "band_NW2"+str(w)+str(d)
        globals()[name1] = [0]
        globals()[name2] = [0]
        globals()[name1_n1] = [0]
        globals()[name1_n2] = [0]
        
        newname1 = "fixed_AD"+str(w)+str(d)
        newname2 = "fixed_SPJ"+str(w)+str(d)
        newname1_n1 = "fixed_NW1"+str(w)+str(d)
        newname1_n2 = "fixed_NW2"+str(w)+str(d)
        globals()[newname1] = [0]
        globals()[newname2] = [0]
        globals()[newname1_n1] = [0]
        globals()[newname1_n2] = [0]
        
        name3 = "stat_AD"+str(w)+str(d)
        name4 = "stat_SPJ"+str(w)+str(d)
        name3_n1 = "stat_NW1"+str(w)+str(d)
        name3_n2 = "stat_NW2"+str(w)+str(d)
        globals()[name3] = [0]
        globals()[name4] = [0]
        globals()[name3_n1] = [0]
        globals()[name3_n2] = [0]
        
        for i in range(200):
            np.random.seed(i)
            u = OU_p(0,30,30*252,ku[w],0,0.0097)
            x = OU_p(0,30,30*252,kx[d],0,0.0998)
            
            #u = OU_AR(0,30,50,ku[w],0,0.0097)
            #x = OU_AR(0,30,50,kx[d],0,0.0998)
            
            u = np.copy(u[::21]) #quaterly
            x = np.copy(x[::21])
            Tnum = x.shape[0]
            
            b0 = 0
            b1 = 1
            y = b0 + b1*x + u
            nu = len(x)
            cons = np.ones((nu,1))
            X = np.hstack((cons,x))
            coeffs = np.linalg.inv(X.transpose().dot(X)).dot(X.transpose()).dot(y)

            X_hat = np.dot(X,coeffs)
            uhat = y - X_hat
            vhat = np.copy(uhat) #G-test
            #vhat = np.copy(X*uhat) #H-test

            R = np.array([[1, 0], [0, 1]])
            r = np.array([[0],[1]])
            
            #####################################################################
            
            vhat2 = np.copy(vhat)
            AD1 = G_test(R,r,coeffs,X,vhat2,c=1,p=4/25,q=2,band="AD")
            #AD1 = H_test(R,r,coeffs,X,vhat,c=1,p=4/25,q=2,band="AD")
            AD_set.append(AD1)
            globals()[name3].append(AD1)
            vhat2 = np.copy(vhat)
            AD_b = AD_band(vhat2)
            globals()[name1].append(AD_b/Tnum)
            
            AD_fixed_cv = 5.9915 + c1*(AD_b/Tnum*5.9915) + c2*((AD_b/Tnum)*(5.9915**2)) + c3*((AD_b/Tnum)*(5.9915**3)) + c4*(((AD_b/Tnum)**2)*5.9915) + c5*(((AD_b/Tnum)**2)*(5.9915**2)) + c6*(((AD_b/Tnum)**2)*(5.9915**3)) + c7*(((AD_b/Tnum)**3)*(5.9915**1)) + c8*(((AD_b/Tnum)**3)*(5.9915**2)) + c9*(((AD_b/Tnum)**3)*(5.9915**3))
            globals()[newname1].append(AD_fixed_cv)
            
            #####################################################################
            vhat3 = np.copy(vhat)
            SPJ1 = G_test(R,r,coeffs,X,vhat3,c=8.5*((1/100)**(21/25)),p=21/25,q=2,band="SPJ")
            #SPJ1 = H_test(R,r,coeffs,X,vhat,c=8.5*((1/100)**(21/25)),p=21/25,q=2,band="SPJ")
            SPJ_set.append(SPJ1)
            globals()[name4].append(SPJ1)
            vhat3 = np.copy(vhat)
            SPJ_b = SPJ_band(vhat3,w=5,z_a=1.96,delta=2,q=2,g=6,c=0.539) #w is tuning parameter
            globals()[name2].append(SPJ_b/Tnum)
            
            SPJ_fixed_cv = 5.9915 + c1*(SPJ_b/Tnum*5.9915) + c2*((SPJ_b/Tnum)*(5.9915**2)) + c3*((SPJ_b/Tnum)*(5.9915**3)) + c4*(((SPJ_b/Tnum)**2)*5.9915) + c5*(((SPJ_b/Tnum)**2)*(5.9915**2)) + c6*(((SPJ_b/Tnum)**2)*(5.9915**3)) + c7*(((SPJ_b/Tnum)**3)*(5.9915**1)) + c8*(((SPJ_b/Tnum)**3)*(5.9915**2)) + c9*(((SPJ_b/Tnum)**3)*(5.9915**3))
            globals()[newname2].append(SPJ_fixed_cv)
            
            #####################################################################
            vhat3 = np.copy(vhat)
            NW1 = G_test(R,r,coeffs,X,vhat3,c=4*((1/100)**(4/25)),p=4/25,q=2,band="NW")
            #NW1 = H_test(R,r,coeffs,X,vhat3,c=4*((1/100)**(4/25)),p=4/25,q=2,band="NW")
            NW1_set.append(NW1)
            globals()[name3_n1].append(NW1)
            vhat3 = np.copy(vhat)
            NW1_b = NW_band(vhat3,c=4*((1/100)**(4/25)),p=4/25,q=2)
            globals()[name1_n1].append(NW1_b/Tnum)
            
            NW1_fixed_cv = 5.9915 + c1*(NW1_b/Tnum*5.9915) + c2*((NW1_b/Tnum)*(5.9915**2)) + c3*((NW1_b/Tnum)*(5.9915**3)) + c4*(((NW1_b/Tnum)**2)*5.9915) + c5*(((NW1_b/Tnum)**2)*(5.9915**2)) + c6*(((NW1_b/Tnum)**2)*(5.9915**3)) + c7*(((NW1_b/Tnum)**3)*(5.9915**1)) + c8*(((NW1_b/Tnum)**3)*(5.9915**2)) + c9*(((NW1_b/Tnum)**3)*(5.9915**3))
            globals()[newname1_n1].append(NW1_fixed_cv)
            
            #####################################################################
            vhat3 = np.copy(vhat)
            NW2 = G_test(R,r,coeffs,X,vhat3,c=8.5*((1/100)**(21/25)),p=21/25,q=2,band="NW")
            #NW2 = H_test(R,r,coeffs,X,vhat3,c=4*((1/100)**(4/25)),p=4/25,q=2,band="NW")
            NW2_set.append(NW2)
            globals()[name3_n2].append(NW2)
            vhat3 = np.copy(vhat)
            NW2_b = NW_band(vhat3,c=8.5*((1/100)**(21/25)),p=21/25,q=2)
            globals()[name1_n2].append(NW2_b/Tnum)
            
            NW2_fixed_cv = 5.9915 + c1*(NW2_b/Tnum*5.9915) + c2*((NW2_b/Tnum)*(5.9915**2)) + c3*((NW2_b/Tnum)*(5.9915**3)) + c4*(((NW2_b/Tnum)**2)*5.9915) + c5*(((NW2_b/Tnum)**2)*(5.9915**2)) + c6*(((NW2_b/Tnum)**2)*(5.9915**3)) + c7*(((NW2_b/Tnum)**3)*(5.9915**1)) + c8*(((NW2_b/Tnum)**3)*(5.9915**2)) + c9*(((NW2_b/Tnum)**3)*(5.9915**3))
            globals()[newname1_n2].append(NW2_fixed_cv)
            
        table_med_AD[w,d] = np.median(AD_set[1:])
        table_med_SPJ[w,d] = np.median(SPJ_set[1:])
        table_mean_AD[w,d] = np.mean(AD_set[1:])
        table_mean_SPJ[w,d] = np.mean(SPJ_set[1:])
        table_min_AD[w,d] = np.min(AD_set[1:])
        table_min_SPJ[w,d] = np.min(SPJ_set[1:])
        table_max_AD[w,d] = np.max(AD_set[1:])
        table_max_SPJ[w,d] = np.max(SPJ_set[1:])
        table_rej_AD[w,d] = sum(x > 5.9915 for x in AD_set[1:])/(i+1)
        table_rej_SPJ[w,d] = sum(y > 5.9915 for y in SPJ_set[1:])/(i+1)
        table_rej_AD_fixed[w,d] = sum(xx>yy for xx, yy in zip(globals()[name3][1:],globals()[newname1][1:]))/(i+1)
        table_rej_SPJ_fixed[w,d] = sum(xx2>yy2 for xx2, yy2 in zip(globals()[name4][1:],globals()[newname2][1:]))/(i+1)
        table_avgband_AD[w,d] = np.mean(globals()[name1][1:])
        table_avgband_SPJ[w,d] = np.mean(globals()[name2][1:])
        table_stdband_AD[w,d] = np.std(globals()[name1][1:])
        table_stdband_SPJ[w,d] = np.std(globals()[name2][1:])
        
        table_med_NW1[w,d] = np.median(NW1_set[1:])
        table_med_NW2[w,d] = np.median(NW2_set[1:])
        table_mean_NW1[w,d] = np.mean(NW1_set[1:])
        table_mean_NW2[w,d] = np.mean(NW2_set[1:])
        table_min_NW1[w,d] = np.min(NW1_set[1:])
        table_min_NW2[w,d] = np.min(NW2_set[1:])
        table_max_NW1[w,d] = np.max(NW1_set[1:])
        table_max_NW2[w,d] = np.max(NW2_set[1:])
        table_rej_NW1[w,d] = sum(x > 5.9915 for x in NW1_set[1:])/(i+1)
        table_rej_NW2[w,d] = sum(y > 5.9915 for y in NW2_set[1:])/(i+1)
        table_rej_NW1_fixed[w,d] = sum(xx>yy for xx, yy in zip(globals()[name3_n1][1:],globals()[newname1_n1][1:]))/(i+1)
        table_rej_NW2_fixed[w,d] = sum(xx2>yy2 for xx2, yy2 in zip(globals()[name3_n2][1:],globals()[newname1_n2][1:]))/(i+1)
        table_avgband_NW1[w,d] = np.mean(globals()[name1_n1][1:])
        table_avgband_NW2[w,d] = np.mean(globals()[name1_n2][1:])
        table_stdband_NW1[w,d] = np.std(globals()[name1_n1][1:])
        table_stdband_NW2[w,d] = np.std(globals()[name1_n2][1:])
        
        
        #AD_set = [0]
        #SPJ_set = [0]

In [181]:
table_mean_AD_1M = np.copy(table_mean_AD)
table_mean_SPJ_1M = np.copy(table_mean_SPJ)
table_mean_NW1_1M = np.copy(table_mean_NW1)
table_mean_NW2_1M = np.copy(table_mean_NW2)

table_rej_AD_1M = np.copy(table_rej_AD)
table_rej_SPJ_1M = np.copy(table_rej_SPJ)
table_rej_NW1_1M = np.copy(table_rej_NW1)
table_rej_NW2_1M = np.copy(table_rej_NW2)

table_rej_AD_fixed_1M = np.copy(table_rej_AD_fixed)
table_rej_SPJ_fixed_1M = np.copy(table_rej_SPJ_fixed)
table_rej_NW1_fixed_1M = np.copy(table_rej_NW1_fixed)
table_rej_NW2_fixed_1M = np.copy(table_rej_NW2_fixed)

table_avgband_AD_1M = np.copy(table_avgband_AD)
table_avgband_SPJ_1M = np.copy(table_avgband_SPJ)
table_avgband_NW1_1M = np.copy(table_avgband_NW1)
table_avgband_NW2_1M = np.copy(table_avgband_NW2)

table_stdband_AD_1M = np.copy(table_stdband_AD)
table_stdband_SPJ_1M = np.copy(table_stdband_SPJ)
table_stdband_NW1_1M = np.copy(table_stdband_NW1)
table_stdband_NW2_1M = np.copy(table_stdband_NW2)

In [182]:
c1 = 0.26880
c2 = 0.98804
c3 = -0.04904
c4 = 5.13641
c5 = -2.37697
c6 = 0.15189
c7 = -2.17925
c8 = 3.06497
c9 = 0.22194
ku = np.linspace(0,6,5) + 0
ku = np.insert(ku, 1, 0.5)
ku = np.delete(ku, 4)
kx = np.linspace(0,6,5) + 0
kx = np.insert(kx, 1, 0.5)
kx = np.delete(kx, 4)
kx[2] = kx[2] + 0.0718
ku[2] = ku[2] + 0.0718
table_mean_AD = np.zeros((5,5))
table_mean_SPJ = np.zeros((5,5))
table_med_AD = np.zeros((5,5))
table_med_SPJ = np.zeros((5,5))
table_min_AD = np.zeros((5,5))
table_min_SPJ = np.zeros((5,5))
table_max_AD = np.zeros((5,5))
table_max_SPJ = np.zeros((5,5))

table_rej_AD = np.zeros((5,5))
table_rej_SPJ = np.zeros((5,5))

table_rej_AD_fixed = np.zeros((5,5))
table_rej_SPJ_fixed = np.zeros((5,5))

table_avgband_AD = np.zeros((5,5))
table_avgband_SPJ = np.zeros((5,5))

table_stdband_AD = np.zeros((5,5))
table_stdband_SPJ = np.zeros((5,5))

table_mean_NW1 = np.zeros((5,5))
table_mean_NW2 = np.zeros((5,5))
table_med_NW1 = np.zeros((5,5))
table_med_NW2 = np.zeros((5,5))
table_min_NW1 = np.zeros((5,5))
table_min_NW2 = np.zeros((5,5))
table_max_NW1 = np.zeros((5,5))
table_max_NW2 = np.zeros((5,5))

table_rej_NW1 = np.zeros((5,5))
table_rej_NW2 = np.zeros((5,5))

table_rej_NW1_fixed = np.zeros((5,5))
table_rej_NW2_fixed = np.zeros((5,5))

table_avgband_NW1 = np.zeros((5,5))
table_avgband_NW2 = np.zeros((5,5))

table_stdband_NW1 = np.zeros((5,5))
table_stdband_NW2 = np.zeros((5,5))


AD_set = [0]
SPJ_set = [0]
NW1_set = [0]
NW2_set = [0]

for w in range(len(ku)):
    for d in range(len(kx)):
        AD_set = [0]
        SPJ_set = [0]
        NW1_set = [0]
        NW2_set = [0]
        
        name1 = "band_AD"+str(w)+str(d)
        name2 = "band_SPJ"+str(w)+str(d)
        name1_n1 = "band_NW1"+str(w)+str(d)
        name1_n2 = "band_NW2"+str(w)+str(d)
        globals()[name1] = [0]
        globals()[name2] = [0]
        globals()[name1_n1] = [0]
        globals()[name1_n2] = [0]
        
        newname1 = "fixed_AD"+str(w)+str(d)
        newname2 = "fixed_SPJ"+str(w)+str(d)
        newname1_n1 = "fixed_NW1"+str(w)+str(d)
        newname1_n2 = "fixed_NW2"+str(w)+str(d)
        globals()[newname1] = [0]
        globals()[newname2] = [0]
        globals()[newname1_n1] = [0]
        globals()[newname1_n2] = [0]
        
        name3 = "stat_AD"+str(w)+str(d)
        name4 = "stat_SPJ"+str(w)+str(d)
        name3_n1 = "stat_NW1"+str(w)+str(d)
        name3_n2 = "stat_NW2"+str(w)+str(d)
        globals()[name3] = [0]
        globals()[name4] = [0]
        globals()[name3_n1] = [0]
        globals()[name3_n2] = [0]
        
        for i in range(200):
            np.random.seed(i)
            u = OU_p(0,30,30*252,ku[w],0,0.0097)
            x = OU_p(0,30,30*252,kx[d],0,0.0998)
            
            #u = OU_AR(0,30,50,ku[w],0,0.0097)
            #x = OU_AR(0,30,50,kx[d],0,0.0998)
            
            u = np.copy(u[::63]) #quaterly
            x = np.copy(x[::63])
            Tnum = x.shape[0]
            
            b0 = 0
            b1 = 1
            y = b0 + b1*x + u
            nu = len(x)
            cons = np.ones((nu,1))
            X = np.hstack((cons,x))
            coeffs = np.linalg.inv(X.transpose().dot(X)).dot(X.transpose()).dot(y)

            X_hat = np.dot(X,coeffs)
            uhat = y - X_hat
            vhat = np.copy(uhat) #G-test
            #vhat = np.copy(X*uhat) #H-test

            R = np.array([[1, 0], [0, 1]])
            r = np.array([[0],[1]])
            
            #####################################################################
            
            vhat2 = np.copy(vhat)
            AD1 = G_test(R,r,coeffs,X,vhat2,c=1,p=4/25,q=2,band="AD")
            #AD1 = H_test(R,r,coeffs,X,vhat,c=1,p=4/25,q=2,band="AD")
            AD_set.append(AD1)
            globals()[name3].append(AD1)
            vhat2 = np.copy(vhat)
            AD_b = AD_band(vhat2)
            globals()[name1].append(AD_b/Tnum)
            
            AD_fixed_cv = 5.9915 + c1*(AD_b/Tnum*5.9915) + c2*((AD_b/Tnum)*(5.9915**2)) + c3*((AD_b/Tnum)*(5.9915**3)) + c4*(((AD_b/Tnum)**2)*5.9915) + c5*(((AD_b/Tnum)**2)*(5.9915**2)) + c6*(((AD_b/Tnum)**2)*(5.9915**3)) + c7*(((AD_b/Tnum)**3)*(5.9915**1)) + c8*(((AD_b/Tnum)**3)*(5.9915**2)) + c9*(((AD_b/Tnum)**3)*(5.9915**3))
            globals()[newname1].append(AD_fixed_cv)
            
            #####################################################################
            vhat3 = np.copy(vhat)
            SPJ1 = G_test(R,r,coeffs,X,vhat3,c=8.5*((1/100)**(21/25)),p=21/25,q=2,band="SPJ")
            #SPJ1 = H_test(R,r,coeffs,X,vhat,c=8.5*((1/100)**(21/25)),p=21/25,q=2,band="SPJ")
            SPJ_set.append(SPJ1)
            globals()[name4].append(SPJ1)
            vhat3 = np.copy(vhat)
            SPJ_b = SPJ_band(vhat3,w=5,z_a=1.96,delta=2,q=2,g=6,c=0.539) #w is tuning parameter
            globals()[name2].append(SPJ_b/Tnum)
            
            SPJ_fixed_cv = 5.9915 + c1*(SPJ_b/Tnum*5.9915) + c2*((SPJ_b/Tnum)*(5.9915**2)) + c3*((SPJ_b/Tnum)*(5.9915**3)) + c4*(((SPJ_b/Tnum)**2)*5.9915) + c5*(((SPJ_b/Tnum)**2)*(5.9915**2)) + c6*(((SPJ_b/Tnum)**2)*(5.9915**3)) + c7*(((SPJ_b/Tnum)**3)*(5.9915**1)) + c8*(((SPJ_b/Tnum)**3)*(5.9915**2)) + c9*(((SPJ_b/Tnum)**3)*(5.9915**3))
            globals()[newname2].append(SPJ_fixed_cv)
            
            #####################################################################
            vhat3 = np.copy(vhat)
            NW1 = G_test(R,r,coeffs,X,vhat3,c=4*((1/100)**(4/25)),p=4/25,q=2,band="NW")
            #NW1 = H_test(R,r,coeffs,X,vhat3,c=4*((1/100)**(4/25)),p=4/25,q=2,band="NW")
            NW1_set.append(NW1)
            globals()[name3_n1].append(NW1)
            vhat3 = np.copy(vhat)
            NW1_b = NW_band(vhat3,c=4*((1/100)**(4/25)),p=4/25,q=2)
            globals()[name1_n1].append(NW1_b/Tnum)
            
            NW1_fixed_cv = 5.9915 + c1*(NW1_b/Tnum*5.9915) + c2*((NW1_b/Tnum)*(5.9915**2)) + c3*((NW1_b/Tnum)*(5.9915**3)) + c4*(((NW1_b/Tnum)**2)*5.9915) + c5*(((NW1_b/Tnum)**2)*(5.9915**2)) + c6*(((NW1_b/Tnum)**2)*(5.9915**3)) + c7*(((NW1_b/Tnum)**3)*(5.9915**1)) + c8*(((NW1_b/Tnum)**3)*(5.9915**2)) + c9*(((NW1_b/Tnum)**3)*(5.9915**3))
            globals()[newname1_n1].append(NW1_fixed_cv)
            
            #####################################################################
            vhat3 = np.copy(vhat)
            NW2 = G_test(R,r,coeffs,X,vhat3,c=8.5*((1/100)**(21/25)),p=21/25,q=2,band="NW")
            #NW2 = H_test(R,r,coeffs,X,vhat3,c=4*((1/100)**(4/25)),p=4/25,q=2,band="NW")
            NW2_set.append(NW2)
            globals()[name3_n2].append(NW2)
            vhat3 = np.copy(vhat)
            NW2_b = NW_band(vhat3,c=8.5*((1/100)**(21/25)),p=21/25,q=2)
            globals()[name1_n2].append(NW2_b/Tnum)
            
            NW2_fixed_cv = 5.9915 + c1*(NW2_b/Tnum*5.9915) + c2*((NW2_b/Tnum)*(5.9915**2)) + c3*((NW2_b/Tnum)*(5.9915**3)) + c4*(((NW2_b/Tnum)**2)*5.9915) + c5*(((NW2_b/Tnum)**2)*(5.9915**2)) + c6*(((NW2_b/Tnum)**2)*(5.9915**3)) + c7*(((NW2_b/Tnum)**3)*(5.9915**1)) + c8*(((NW2_b/Tnum)**3)*(5.9915**2)) + c9*(((NW2_b/Tnum)**3)*(5.9915**3))
            globals()[newname1_n2].append(NW2_fixed_cv)
            
        table_med_AD[w,d] = np.median(AD_set[1:])
        table_med_SPJ[w,d] = np.median(SPJ_set[1:])
        table_mean_AD[w,d] = np.mean(AD_set[1:])
        table_mean_SPJ[w,d] = np.mean(SPJ_set[1:])
        table_min_AD[w,d] = np.min(AD_set[1:])
        table_min_SPJ[w,d] = np.min(SPJ_set[1:])
        table_max_AD[w,d] = np.max(AD_set[1:])
        table_max_SPJ[w,d] = np.max(SPJ_set[1:])
        table_rej_AD[w,d] = sum(x > 5.9915 for x in AD_set[1:])/(i+1)
        table_rej_SPJ[w,d] = sum(y > 5.9915 for y in SPJ_set[1:])/(i+1)
        table_rej_AD_fixed[w,d] = sum(xx>yy for xx, yy in zip(globals()[name3][1:],globals()[newname1][1:]))/(i+1)
        table_rej_SPJ_fixed[w,d] = sum(xx2>yy2 for xx2, yy2 in zip(globals()[name4][1:],globals()[newname2][1:]))/(i+1)
        table_avgband_AD[w,d] = np.mean(globals()[name1][1:])
        table_avgband_SPJ[w,d] = np.mean(globals()[name2][1:])
        table_stdband_AD[w,d] = np.std(globals()[name1][1:])
        table_stdband_SPJ[w,d] = np.std(globals()[name2][1:])
        
        table_med_NW1[w,d] = np.median(NW1_set[1:])
        table_med_NW2[w,d] = np.median(NW2_set[1:])
        table_mean_NW1[w,d] = np.mean(NW1_set[1:])
        table_mean_NW2[w,d] = np.mean(NW2_set[1:])
        table_min_NW1[w,d] = np.min(NW1_set[1:])
        table_min_NW2[w,d] = np.min(NW2_set[1:])
        table_max_NW1[w,d] = np.max(NW1_set[1:])
        table_max_NW2[w,d] = np.max(NW2_set[1:])
        table_rej_NW1[w,d] = sum(x > 5.9915 for x in NW1_set[1:])/(i+1)
        table_rej_NW2[w,d] = sum(y > 5.9915 for y in NW2_set[1:])/(i+1)
        table_rej_NW1_fixed[w,d] = sum(xx>yy for xx, yy in zip(globals()[name3_n1][1:],globals()[newname1_n1][1:]))/(i+1)
        table_rej_NW2_fixed[w,d] = sum(xx2>yy2 for xx2, yy2 in zip(globals()[name3_n2][1:],globals()[newname1_n2][1:]))/(i+1)
        table_avgband_NW1[w,d] = np.mean(globals()[name1_n1][1:])
        table_avgband_NW2[w,d] = np.mean(globals()[name1_n2][1:])
        table_stdband_NW1[w,d] = np.std(globals()[name1_n1][1:])
        table_stdband_NW2[w,d] = np.std(globals()[name1_n2][1:])
        
        
        #AD_set = [0]
        #SPJ_set = [0]

In [183]:
table_mean_AD_1Q = np.copy(table_mean_AD)
table_mean_SPJ_1Q = np.copy(table_mean_SPJ)
table_mean_NW1_1Q = np.copy(table_mean_NW1)
table_mean_NW2_1Q = np.copy(table_mean_NW2)

table_rej_AD_1Q = np.copy(table_rej_AD)
table_rej_SPJ_1Q = np.copy(table_rej_SPJ)
table_rej_NW1_1Q = np.copy(table_rej_NW1)
table_rej_NW2_1Q = np.copy(table_rej_NW2)

table_rej_AD_fixed_1Q = np.copy(table_rej_AD_fixed)
table_rej_SPJ_fixed_1Q = np.copy(table_rej_SPJ_fixed)
table_rej_NW1_fixed_1Q = np.copy(table_rej_NW1_fixed)
table_rej_NW2_fixed_1Q = np.copy(table_rej_NW2_fixed)

table_avgband_AD_1Q = np.copy(table_avgband_AD)
table_avgband_SPJ_1Q = np.copy(table_avgband_SPJ)
table_avgband_NW1_1Q = np.copy(table_avgband_NW1)
table_avgband_NW2_1Q = np.copy(table_avgband_NW2)

table_stdband_AD_1Q = np.copy(table_stdband_AD)
table_stdband_SPJ_1Q = np.copy(table_stdband_SPJ)
table_stdband_NW1_1Q = np.copy(table_stdband_NW1)
table_stdband_NW2_1Q = np.copy(table_stdband_NW2)

In [ ]:
##########################

In [ ]:
for w in range(5):
    for d in range(5):
        y = chi[0:,w,d]
        #yy = chi2[0:,w,d]
        x = np.linspace(0,2,2) + 0
        #x = x[1:]
        plt.rcParams["figure.figsize"]=(9, 6)
        plt.plot(x, y, label="Chi-square")
        #plt.plot(x, yy, label="Fixed-b")
        y2 = np.zeros((y.shape)) +0.05
        plt.plot(x,y2,'r--',label="0.05 level")
        #plt.figure(figsize=(100, 100))
        plt.xticks(np.arange(0, max(x), 1))
        plt.xlabel("Bandwidth", fontsize=15)
        plt.ylabel("Rejection Probabilities", fontsize=15)
        plt.title('$k_{u}$=%1.1f, ' %ku[w] + '$k_{x}$=%1.1f, ' %kx[d] +"(Sample size = 7560)", fontsize=15)
        plt.legend(loc="upper center", fontsize=12)
        plt.show()
        plt.clf()

In [207]:
chi_rej_AD = np.stack([table_rej_AD_D,table_rej_AD_2D,table_rej_AD_5D,table_rej_AD_1M,table_rej_AD_1Q])
chi_rej_SPJ = np.stack([table_rej_SPJ_D,table_rej_SPJ_2D,table_rej_SPJ_5D,table_rej_SPJ_1M,table_rej_SPJ_1Q])
chi_rej_NW1 = np.stack([table_rej_NW1_D,table_rej_NW1_2D,table_rej_NW1_5D,table_rej_NW1_1M,table_rej_NW1_1Q])
chi_rej_NW2 = np.stack([table_rej_NW2_D,table_rej_NW2_2D,table_rej_NW2_5D,table_rej_NW2_1M,table_rej_NW2_1Q])

In [208]:
chi_mean_AD = np.stack([table_mean_AD_D,table_mean_AD_2D,table_mean_AD_5D,table_mean_AD_1M,table_mean_AD_1Q])
chi_mean_SPJ = np.stack([table_mean_SPJ_D,table_mean_SPJ_2D,table_mean_SPJ_5D,table_mean_SPJ_1M,table_mean_SPJ_1Q])
chi_mean_NW1 = np.stack([table_mean_NW1_D,table_mean_NW1_2D,table_mean_NW1_5D,table_mean_NW1_1M,table_mean_NW1_1Q])
chi_mean_NW2 = np.stack([table_mean_NW2_D,table_mean_NW2_2D,table_mean_NW2_5D,table_mean_NW2_1M,table_mean_NW2_1Q])

In [209]:
chi_rej_fixed_AD = np.stack([table_rej_AD_fixed_D,table_rej_AD_fixed_2D,table_rej_AD_fixed_5D,table_rej_AD_fixed_1M,table_rej_AD_fixed_1Q])
chi_rej_fixed_SPJ = np.stack([table_rej_SPJ_fixed_D,table_rej_SPJ_fixed_2D,table_rej_SPJ_fixed_5D,table_rej_SPJ_fixed_1M,table_rej_SPJ_fixed_1Q])
chi_rej_fixed_NW1 = np.stack([table_rej_NW1_D,table_rej_NW1_2D,table_rej_NW1_5D,table_rej_NW1_1M,table_rej_NW1_1Q])
chi_rej_fixed_NW2 = np.stack([table_rej_Nw2_fixed_D,table_rej_NW2_fixed_2D,table_rej_NW2_fixed_5D,table_rej_NW2_fixed_1M,table_rej_NW2_fixed_1Q])

In [210]:
chi_avgband_AD = np.stack([table_avgband_AD_D,table_avgband_AD_2D,table_avgband_AD_5D,table_avgband_AD_1M,table_avgband_AD_1Q])
chi_avgband_SPJ = np.stack([table_avgband_SPJ_D,table_avgband_SPJ_2D,table_avgband_SPJ_5D,table_avgband_SPJ_1M,table_avgband_SPJ_1Q])
chi_avgband_NW1 = np.stack([table_avgband_NW1_D,table_avgband_NW1_2D,table_avgband_NW1_5D,table_avgband_NW1_1M,table_avgband_NW1_1Q])
chi_avgband_NW2 = np.stack([table_avgband_NW2_D,table_avgband_NW2_2D,table_avgband_NW2_5D,table_avgband_NW2_1M,table_avgband_NW2_1Q])

In [217]:
x

array([0.  , 0.25, 0.5 , 0.75, 1.  ])

In [219]:
np.array([1/252, 1/126, 1/(252/5), 1/12, 1/4])

array([0.00396825, 0.00793651, 0.01984127, 0.08333333, 0.25      ])

In [271]:
plt.clf()
for w in range(5):
    for d in range(5):
        y = chi_mean_AD[0:,w,d]
        y2 = chi_mean_SPJ[0:,w,d]
        y3 = chi_mean_NW1[0:,w,d]
        y4 = chi_mean_NW2[0:,w,d]
        #yy = chi2[0:,w,d]
        x = np.array([1/252, 1/126, 1/(252/5), 1/12, 1/4])
        #x = x[1:]
        plt.rcParams["figure.figsize"]=(9, 6)
        plt.plot(x, y, 'b-.', label="AD")
        plt.plot(x, y2, label="SPJ", color='orange')
        plt.plot(x, y3, 'g-', label="NW")
        plt.plot(x, y4, 'r--', label="NW_Tune")
        #plt.plot(x, yy, label="Fixed-b")
        yline = np.zeros((y.shape)) +0.05
        #plt.plot(x,yline,'k--',label="0.05 level")
        #plt.figure(figsize=(100, 100))
        plt.xticks(np.arange(0, max(x), 0.04))
        plt.ylim(0, max(y3))
        plt.yticks(np.arange(0, max(max(y3),30), np.round(max(max(y3),30)/6,0)))
        plt.xlabel("Frequency", fontsize=15)
        plt.ylabel("Average of Wald", fontsize=15)
        plt.title('$k_{u}$=%1.1f, ' %ku[w] + '$k_{x}$=%1.1f, ' %kx[d] +"(Average of Wald)", fontsize=15)
        plt.legend(loc="upper center", fontsize=12)
        #plt.show()
        plt.savefig("aavgplot%d"%w +"%d.eps"%d,dpi=600)
        plt.clf()

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript back

<Figure size 648x432 with 0 Axes>

In [272]:
plt.clf()
for w in range(5):
    for d in range(5):
        y = chi_rej_AD[0:,w,d]
        y2 = chi_rej_SPJ[0:,w,d]
        y3 = chi_rej_NW1[0:,w,d]
        y4 = chi_rej_NW2[0:,w,d]
        #yy = chi2[0:,w,d]
        x = np.array([1/252, 1/126, 1/(252/5), 1/12, 1/4])
        #x = x[1:]
        plt.rcParams["figure.figsize"]=(9, 6)
        plt.plot(x, y, 'b-.', label="AD")
        plt.plot(x, y2, label="SPJ", color='orange')
        plt.plot(x, y3, 'g-', label="NW")
        plt.plot(x, y4, 'r--', label="NW_Tune")
        #plt.plot(x, yy, label="Fixed-b")
        yline = np.zeros((y.shape)) +0.05
        plt.plot(x,yline,'k--',label="0.05 level")
        #plt.figure(figsize=(100, 100))
        plt.xticks(np.arange(0, max(x), 0.04))
        plt.yticks(np.arange(0, 1, 0.1))
        plt.xlabel("Frequency", fontsize=15)
        plt.ylabel("Rejection Probabilities", fontsize=15)
        plt.title('$k_{u}$=%1.1f, ' %ku[w] + '$k_{x}$=%1.1f, ' %kx[d] +"(Rejection Probs)", fontsize=15)
        plt.legend(loc="upper center", fontsize=12)
        #plt.show()
        plt.savefig("rrejplot%d"%w +"%d.eps"%d,dpi=600)
        plt.clf()

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript back

<Figure size 648x432 with 0 Axes>

In [273]:
plt.clf()
for w in range(5):
    for d in range(5):
        y = chi_rej_fixed_AD[0:,w,d]
        y2 = chi_rej_fixed_SPJ[0:,w,d]
        y3 = chi_rej_fixed_NW1[0:,w,d]
        y4 = chi_rej_fixed_NW2[0:,w,d]
        #yy = chi2[0:,w,d]
        x = np.array([1/252, 1/126, 1/(252/5), 1/12, 1/4])
        #x = x[1:]
        plt.rcParams["figure.figsize"]=(9, 6)
        plt.plot(x, y, 'b-.', label="AD")
        plt.plot(x, y2, label="SPJ", color='orange')
        plt.plot(x, y3, 'g-', label="NW")
        plt.plot(x, y4, 'r--', label="NW_Tune")
        #plt.plot(x, yy, label="Fixed-b")
        yline = np.zeros((y.shape)) +0.05
        plt.plot(x,yline,'k--',label="0.05 level")
        #plt.figure(figsize=(100, 100))
        plt.xticks(np.arange(0, max(x), 0.04))
        plt.yticks(np.arange(0, 1, 0.1))
        plt.xlabel("Frequency", fontsize=15)
        plt.ylabel("Rejection Probabilities", fontsize=15)
        plt.title('$k_{u}$=%1.1f, ' %ku[w] + '$k_{x}$=%1.1f, ' %kx[d] +"(Rejection Probs, fixed-b)", fontsize=15)
        plt.legend(loc="upper center", fontsize=12)
        plt.savefig("rrejfplot%d"%w +"%d.eps"%d,dpi=600)
        plt.clf()

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript back

<Figure size 648x432 with 0 Axes>

In [274]:
plt.clf()
for w in range(5):
    for d in range(5):
        y = chi_avgband_AD[0:,w,d]
        y2 = chi_avgband_SPJ[0:,w,d]
        y3 = chi_avgband_NW1[0:,w,d]
        y4 = chi_avgband_NW2[0:,w,d]
        #yy = chi2[0:,w,d]
        x = np.array([1/252, 1/126, 1/(252/5), 1/12, 1/4])
        #x = x[1:]
        plt.rcParams["figure.figsize"]=(9, 6)
        plt.plot(x, y, 'b-.', label="AD")
        plt.plot(x, y2, label="SPJ", color='orange')
        plt.plot(x, y3, 'g-', label="NW")
        plt.plot(x, y4, 'r--', label="NW_Tune")
        #plt.plot(x, yy, label="Fixed-b")
        yline = np.zeros((y.shape)) +0.05
        #plt.plot(x,yline,'k--',label="0.05 level")
        #plt.figure(figsize=(100, 100))
        plt.xticks(np.arange(0, max(x), 0.04))
        plt.yticks(np.arange(0, 1, 0.1))
        plt.xlabel("Frequency", fontsize=15)
        plt.ylabel("Average of bandwidths", fontsize=15)
        plt.title('$k_{u}$=%1.1f, ' %ku[w] + '$k_{x}$=%1.1f, ' %kx[d] +"(Average of bandwidths)", fontsize=15)
        plt.legend(loc="upper center", fontsize=12)
        
        #plt.show()
        plt.savefig("bbandplot%d"%w +"%d.eps"%d,dpi=600)
        plt.clf()

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript back

<Figure size 648x432 with 0 Axes>